In [19]:
import pandas as pd
pd.options.mode.chained_assignment = None  # This is to handle an error of pandas DataFrame copy, it's optional.
import requests
import psycopg2
from clickhouse_driver import Client
from credentials import dwh_user, dwh_pass, click_user, click_pass
import datetime
import os

In [20]:
startingC = datetime.datetime.now()

In [21]:
# User Input

# Insert dates (year, month, day, hour, minute, second)
# Format should be UTC time
# Maximum report length is 41 day (after it historic prices will be incorrect)
start_date = datetime.datetime(2021, 7, 18, 21, 0, 0) # Monday
end_date = datetime.datetime(2021, 7, 25, 21, 0, 0) # Sunday

mode = 'weekly' # Can be: weekly, daily, no, retro
binance_mode = 'db' # Can be: csv, db # This is where trades are downloaded from

# Other inputs and notes
extreme = 100 # После подсчёта PnL появится дополнительная таблица с парами, где PnL больше этого значения
# В финальной таблице Exposure будут исключены L-tokens из подсчёта

In [22]:
# Connect to DWH
def run_dwh_sql(sql_query):
    conn = psycopg2.connect(host='135.181.61.116', user=dwh_user,password=dwh_pass, dbname='postgres',port='5432')
    cur = conn.cursor()
    cur.execute("SET statement_timeout = 0")
    data = pd.read_sql(sql_query, conn)
    conn.close()
    return data

client = Client("95.217.178.73", user=click_user, password=click_pass, database="default")

# Queries:
# Transactions (deposits and withdrawals)
def transactions_latoken(start_date, end_date):
    sql_query = f"""
    select vtmtl."user"            as user_id,
       mvamcl.tag              as currency_tag,
       vtmtl.transferred_funds as value,
       vtmtl.usd_value,
       vtmtl.transaction_fee   as fee_value,
       vtmtl.status,
       vtmtl.__update_datetime as updated_at,
       vtmtl.type              as transaction_type
    from view_transaction_manager_transaction_latest vtmtl
         left join materialized_view_transaction_manager_currency_binding_latest mvtmcbl
                   on vtmtl.currency_binding_id = mvtmcbl.id
         left join materialized_view_asset_manager_currency_latest mvamcl
                   on mvtmcbl.currency = mvamcl.id
    where vtmtl.__update_datetime >= '{start_date}' and vtmtl.__update_datetime <= '{end_date}'
    and vtmtl.status = 'CONFIRMED'
    and vtmtl."user" in ('2b58cccc-5fbd-4154-9e73-a009d7c145c9',
                            '8fdd328f-47cc-4705-bf45-9a4f20314b87',
                            'ddee6a46-3bbd-4d39-b561-c028e618d8ff',
                            'e6fa4889-93ab-4f9b-a773-39f079180706',
                            'fbd3f154-dd62-419d-9d59-bfdc327e231a',
                           '708b9327-1c3d-4841-8ee4-85acfbbc3159');
     """
    return run_dwh_sql(sql_query)

# Transafers
def transfers_latoken(start_date, end_date):
    sql_query = f"""
    select vamtl.type            as type,
       vamtl.status              as status,
       mvamal.user               as from_user_id,
       vamtl.from_account_id     as from_account_id,
       mvamcl.tag                as from_asset_tag,
       mvamcl.name               as from_asset_name,
       mvamal2.user              as to_user_id,
       vamtl.to_account_id       as to_account_id,
       mvamcl2.tag               as to_asset_tag,
       mvamcl2.name              as to_asset_name,
       vamtl.transferring_amount as amount,
       vamtl.usd_value           as amount_usd,
       vamtl.__create_datetime   as created_at,
       vamtl.__update_datetime   as updated_at
    from view_account_manager_transfer_latest vamtl
         inner join materialized_view_account_manager_account_latest as mvamal
                    on vamtl.from_account_id = mvamal.id
         inner join materialized_view_account_manager_account_latest as mvamal2
                    on vamtl.to_account_id = mvamal2.id
         inner join materialized_view_asset_manager_currency_latest mvamcl
                    on mvamal.currency = mvamcl.id
         inner join materialized_view_asset_manager_currency_latest mvamcl2
                    on mvamal2.currency = mvamcl2.id
    where vamtl.__update_datetime >= '{start_date}'
    and vamtl.__update_datetime <= '{end_date}'
    and vamtl.type not in ('DEPOSIT_SPOT','WITHDRAW_SPOT')
    and vamtl.status = 'COMPLETED'
    and (mvamal.user in ('2b58cccc-5fbd-4154-9e73-a009d7c145c9',
                            '8fdd328f-47cc-4705-bf45-9a4f20314b87',
                            'ddee6a46-3bbd-4d39-b561-c028e618d8ff',
                            'e6fa4889-93ab-4f9b-a773-39f079180706',
                            'fbd3f154-dd62-419d-9d59-bfdc327e231a',
                           '708b9327-1c3d-4841-8ee4-85acfbbc3159')
    or mvamal2.user in ('2b58cccc-5fbd-4154-9e73-a009d7c145c9',
                            '8fdd328f-47cc-4705-bf45-9a4f20314b87',
                            'ddee6a46-3bbd-4d39-b561-c028e618d8ff',
                            'e6fa4889-93ab-4f9b-a773-39f079180706',
                            'fbd3f154-dd62-419d-9d59-bfdc327e231a',
                           '708b9327-1c3d-4841-8ee4-85acfbbc3159'));
    """
    return run_dwh_sql(sql_query)

# Balances current for rotation bot, la-liquidity, margin, leverage, lakuone accounts 
def balances_current(end_date):
    sql_query = f"""
    select anyLast(date), anyLast(datetime), investor_id, asset, account_type, anyLast(available), anyLast(locked), (anyLast(total)-anyLast(borrowed)), exchange, anyLast(total_rfc)
    from (select * from bot_trading_balances_arch order by datetime)
    where datetime >= '{(end_date - datetime.timedelta(seconds=20))}' and datetime <= '{end_date}'
    group by exchange, investor_id, account_type, asset;
    """
    df = pd.DataFrame(client.execute(sql_query))
    df = df.rename({0: 'date', 1: 'datetime', 2: 'investor_id', 3: 'currency', 4: 'account_type', 
                    5: 'available', 6: 'locked', 7: 'total', 8: 'exchange', 9: 'total_rfc'}, axis=1)
    return df

# Balances current for a floating bot
def balances_current_floating(end_date):
    sql_query = f"""
    select anyLast(date), anyLast(datetime), investor_id, asset, account_type, anyLast(available), anyLast(locked), anyLast(total), 'LATOKEN' as exchange, anyLast(total_rfc)
    from (select * from floating_bot_trading_balances_arch order by datetime)
    where datetime >= '{(end_date - datetime.timedelta(seconds=20))}' and datetime <= '{end_date}'
    group by exchange, investor_id, account_type, asset;
    """
    df = pd.DataFrame(client.execute(sql_query))
    df = df.rename({0: 'date', 1: 'datetime', 2: 'investor_id', 3: 'currency', 4: 'account_type', 
                    5: 'available', 6: 'locked', 7: 'total', 8: 'exchange', 9: 'total_rfc'}, axis=1)
    return df

# Trades
def trades_latoken(start_date, end_date):
    sql_query = f"""
    select quote, currency, sum(quantity) as quantity, sum(cost) as cost, direction, trader, status
    from(
        select quote, currency, sum(quantity) as quantity, sum(cost) as cost, direction, maker_trader as trader, 'maker' as status
        from view_market_aggregator_trade
        where __update_datetime >= '{start_date}' and __update_datetime <= '{end_date}'
        and maker_trader in ('2b58cccc-5fbd-4154-9e73-a009d7c145c9',
                            '8fdd328f-47cc-4705-bf45-9a4f20314b87',
                            'ddee6a46-3bbd-4d39-b561-c028e618d8ff',
                            'e6fa4889-93ab-4f9b-a773-39f079180706',
                            'fbd3f154-dd62-419d-9d59-bfdc327e231a',
                           '708b9327-1c3d-4841-8ee4-85acfbbc3159')
        and taker_trader!=maker_trader
        group by quote, currency, direction, trader, status
        union all
        select quote, currency, sum(quantity) as quantity, sum(cost) as cost, direction, taker_trader as trader, 'taker' as status
        from view_market_aggregator_trade
        where __update_datetime >= '{start_date}' and __update_datetime <= '{end_date}'
        and taker_trader in ('2b58cccc-5fbd-4154-9e73-a009d7c145c9',
                            '8fdd328f-47cc-4705-bf45-9a4f20314b87',
                            'ddee6a46-3bbd-4d39-b561-c028e618d8ff',
                            'e6fa4889-93ab-4f9b-a773-39f079180706',
                            'fbd3f154-dd62-419d-9d59-bfdc327e231a',
                           '708b9327-1c3d-4841-8ee4-85acfbbc3159')
        and taker_trader!=maker_trader
        group by quote, currency, direction, trader, status) w
    group by quote, currency, direction, trader, status;
    """
    return run_dwh_sql(sql_query)

# Binance trades (spot+margin) start_date + datetime.timedelta(hours=1), (end_date + datetime.timedelta(hours=1))
def trades_binance(start_date, end_date):
    sql_query = f"""
    select symbol, sum(last_currency_qty), sum(last_quote_qty), side, 'Binance' as trader, 'maker' as status
    from binance_spot_trades
    where order_trade_time >= '{start_date}' and order_trade_time <= '{end_date}'
    group by symbol, side
    order by symbol desc;
    """
    df = pd.DataFrame(client.execute(sql_query))
    return df

# Getting Binance fees start_date + datetime.timedelta(hours=1), (end_date + datetime.timedelta(hours=1))
def binance_fees(start_date, end_date):
    sql_query = f"""
    select commission_asset, sum(commission)
    from binance_spot_trades
    where order_trade_time >= '{start_date}' and order_trade_time <= '{end_date}'
    group by commission_asset;
    """
    df = pd.DataFrame(client.execute(sql_query))
    return df

# Getting currencies instead of ids
def currencies(x):
    sql_query = f"""
    select id, tag
    from view_asset_manager_currency
    where id in {x};
    """
    return run_dwh_sql(sql_query)

# Getting ids instead of currency tags
def ids(x):
    sql_query = f"""
    select tag, id
    from view_asset_manager_currency
    where tag in {x};
    """
    return run_dwh_sql(sql_query)

# Getting transfer adjustments
def transfer_adjustments():
    sql_query = f"""
    select vamtl.__update_datetime as updated,
        mvamcl.tag                as currency_tag,
       sum(vamtl.transferring_amount) as amount
    from view_account_manager_transfer_latest vamtl
         inner join materialized_view_account_manager_account_latest as mvamal
                    on vamtl.from_account_id = mvamal.id
         inner join materialized_view_account_manager_account_latest as mvamal2
                    on vamtl.to_account_id = mvamal2.id
         inner join materialized_view_asset_manager_currency_latest mvamcl
                    on mvamal.currency = mvamcl.id
    where vamtl.type = 'BALANCE_ADJUSTMENT'
    and vamtl.status = 'COMPLETED'
    and mvamal.user = '00000000-0000-0000-0000-000000000000'
    and mvamal2.user in ('8fdd328f-47cc-4705-bf45-9a4f20314b87','ddee6a46-3bbd-4d39-b561-c028e618d8ff')
    group by currency_tag, updated;
    """
    return run_dwh_sql(sql_query)

def transaction_adjustments():
    sql_query = f"""
    select vtmtl.__update_datetime as updated,
       mvamcl.tag              as currency_tag,
       sum(vtmtl.transferred_funds) as amount
    from view_transaction_manager_transaction_latest vtmtl
         left join materialized_view_transaction_manager_currency_binding_latest mvtmcbl
                   on vtmtl.currency_binding_id = mvtmcbl.id
         left join materialized_view_transaction_manager_currency_provider_latest mvtmcpl
                   on mvtmcbl.currency_provider_id = mvtmcpl.id
         left join materialized_view_asset_manager_currency_latest mvamcl
                   on mvtmcbl.currency = mvamcl.id
    where vtmtl.status = 'CONFIRMED'
    and vtmtl.type = 'DEPOSIT'
    and (vtmtl.transaction_hash like '0000000000000000000000000000000000000000000000000000000000000000#%' or
    vtmtl.transaction_hash like '%orrow%')
    group by currency_tag, updated;
    """
    return run_dwh_sql(sql_query)

# Getting transfer adjustments
def transfer_adjustments_other():
    sql_query = f"""
    select vamtl.__update_datetime as updated,
        mvamcl.tag                as currency_tag,
        mvamal2.user              as account,
       sum(vamtl.transferring_amount) as amount
    from view_account_manager_transfer_latest vamtl
         inner join materialized_view_account_manager_account_latest as mvamal
                    on vamtl.from_account_id = mvamal.id
         inner join materialized_view_account_manager_account_latest as mvamal2
                    on vamtl.to_account_id = mvamal2.id
         inner join materialized_view_asset_manager_currency_latest mvamcl
                    on mvamal.currency = mvamcl.id
    where vamtl.type = 'BALANCE_ADJUSTMENT'
    and vamtl.status = 'COMPLETED'
    and mvamal.user = '00000000-0000-0000-0000-000000000000'
    and mvamal2.user in ('e6fa4889-93ab-4f9b-a773-39f079180706',
                            'fbd3f154-dd62-419d-9d59-bfdc327e231a',
                           '708b9327-1c3d-4841-8ee4-85acfbbc3159')
    group by currency_tag, updated, account;
    """
    return run_dwh_sql(sql_query)

# Binance transactions (deposits + withdrawals) start_date + datetime.timedelta(hours=1), end_date + datetime.timedelta(hours=1)
def binance_transactions(start_date, end_date):
    sql_query = f"""
    select date_time, asset, amount, type, fee
    from binance_withdrawal_deposit
    where date_time >= '{start_date}' 
    and date_time <= '{end_date}'
    and (status = 'Success' or status = 'Completed');
    """
    df = pd.DataFrame(client.execute(sql_query), columns = ['time', 'currency', 'amount', 'type', 'fee'])
    return df

# Active currencies by id
def active_currencies(currencies):
    sql_query = f"""
    select currency_id as currency, status as status, row_number() over(partition by currency_id order by status desc) as row
    from view_asset_manager_pair
    where currency_id in {currencies}
    group by currency_id, status;
    """
    return run_dwh_sql(sql_query)

# Getting Binance margin fees
def binance_margin_fees(start_date, end_date):
    sql_query = f"""
    select time, asset, sum(interest), sum(interest_rfc)
    from binance_margin_repays
    where time >= '{start_date}' and time <= '{(end_date)}'
    and interest != 0
    group by time, asset;
    """
    df = pd.DataFrame(client.execute(sql_query), columns = ['time', 'asset', 'interest', 'interest_rfc'])
    return df

In [23]:
account_type = {'2b58cccc-5fbd-4154-9e73-a009d7c145c9': '65127',
             '8fdd328f-47cc-4705-bf45-9a4f20314b87': 'leverage',
             'Binance': 'binance_trading',
             'ddee6a46-3bbd-4d39-b561-c028e618d8ff': 'margin',
               'e6fa4889-93ab-4f9b-a773-39f079180706': 'floating',
               'fbd3f154-dd62-419d-9d59-bfdc327e231a': 'rotation',
               '708b9327-1c3d-4841-8ee4-85acfbbc3159': 'laliquidity'}


token_tier = {'USDL': 'L-tokens', 
            'RUPL': 'L-tokens',
            'ARSL': 'L-tokens',
            'GBPL': 'L-tokens',
            'BRLL': 'L-tokens',
            'EURL': 'L-tokens',
            'CHFL': 'L-tokens',
            'GOLDL': 'L-tokens',
            'AAPLL': 'L-tokens',
            'BABAL': 'L-tokens',
            'KGSL': 'L-tokens',
            'USD': 'Fiat',
              'LA': 'LA',
              'ETH': 'tier 1',
            'BTC': 'tier 1',
            'USDT': 'tier 1',
            'USDC': 'tier 1',
              'TRX': 'tier 2',
            'XLM': 'tier 2',
            'LTC': 'tier 2',
            'BCH': 'tier 2',
            'XRP': 'tier 2',
            'DASH': 'tier 2'}  # All other tokens are tier 3

In [24]:
# This section: Creating 'trades' table
# Getting LATOKEN trades data
latoken_trades = trades_latoken(start_date, end_date)

# Converting currency ids into tickers
latoken_currency_list = set(latoken_trades.currency.unique())
latoken_quote_list = set(latoken_trades.quote.unique())
latoken_currency_tuple = tuple(set(latoken_currency_list|latoken_quote_list))

latoken_currencies = currencies(latoken_currency_tuple)
latoken_currencies.index = latoken_currencies.id
latoken_currencies = latoken_currencies.drop(columns=['id'])
latoken_currencies = latoken_currencies.to_dict()['tag']  # {id: tag} excludes Binance currencies not on LATOKEN
currencies_ids = {y:x for x,y in latoken_currencies.items()}  # {tag: id} excludes Binance currencies not on LATOKEN

latoken_trades['currency1'] = latoken_trades.currency.apply(lambda x: latoken_currencies[x])
latoken_trades['quote1'] = latoken_trades.quote.apply(lambda x: latoken_currencies[x])
latoken_trades['pair'] = latoken_trades.currency1 + latoken_trades.quote1


# This section: Getting BINANCE trades
if binance_mode == 'db':
    binance_trades = trades_binance(start_date, end_date)
    binance_trades = binance_trades.rename({0: 'pair', 1: 'quantity', 2: 'cost', 3: 'direction', 4: 'trader', 5: 'status'}, axis=1)
elif binance_mode == 'csv':
    spot = pd.read_excel(f'{os.getcwd()}/Spot.xlsx')
    margin = pd.read_excel(f'{os.getcwd()}/Margin.xlsx')
    binance_trades = pd.concat([spot, margin])
    binance_trades = binance_trades.rename({'Fee Coin': 'fee_asset'}, axis=1)
    binanceFees = binance_trades.groupby(['fee_asset'])['Fee'].sum()
    binance_trades = binance_trades.groupby(['Market', 'Type'])[['Amount', 'Total']].sum().reset_index()
    binance_trades['trader'] = 'Binance'
    binance_trades['status'] = 'maker'
    binance_trades = binance_trades.rename({'Market': 'pair', 'Amount': 'quantity', 'Total': 'cost', 'Type': 'direction'}, axis=1)
else:
    print('I do not know this type of Binance trades uploading')
    exit()

binanceUniquePairs = list(binance_trades.pair.unique())
binanceQuoteIDs = dict()
binanceQuoteTag = dict()
binanceCurTag = dict()
for i in range(len(binanceUniquePairs)):
    if binanceUniquePairs[i].endswith('BTC'):
        binanceQuoteIDs[binanceUniquePairs[i]] = '92151d82-df98-4d88-9a4d-284fa9eca49f'
        binanceQuoteTag[binanceUniquePairs[i]] = 'BTC'
        binanceCurTag[binanceUniquePairs[i]] = binanceUniquePairs[i][:-3]
    elif binanceUniquePairs[i].endswith('ETH'):
        binanceQuoteIDs[binanceUniquePairs[i]] = '620f2019-33c0-423b-8a9d-cde4d7f8ef7f'
        binanceQuoteTag[binanceUniquePairs[i]] = 'ETH'
        binanceCurTag[binanceUniquePairs[i]] = binanceUniquePairs[i][:-3]
    elif binanceUniquePairs[i].endswith('USDT'):
        binanceQuoteIDs[binanceUniquePairs[i]] = '0c3a106d-bde3-4c13-a26e-3fd2394529e5'
        binanceQuoteTag[binanceUniquePairs[i]] = 'USDT'
        binanceCurTag[binanceUniquePairs[i]] = binanceUniquePairs[i][:-4]
    elif binanceUniquePairs[i].endswith('EOS'):
        binanceQuoteIDs[binanceUniquePairs[i]] = 'd286007b-03eb-454e-936f-296c4c6e3be9'
        binanceQuoteTag[binanceUniquePairs[i]] = 'EOS'
        binanceCurTag[binanceUniquePairs[i]] = binanceUniquePairs[i][:-3]    
    elif binanceUniquePairs[i].endswith('TRX'):
        binanceQuoteIDs[binanceUniquePairs[i]] = '34629b4b-753c-4537-865f-4b62ff1a31d6'
        binanceQuoteTag[binanceUniquePairs[i]] = 'TRX'
        binanceCurTag[binanceUniquePairs[i]] = binanceUniquePairs[i][:-3]
    elif binanceUniquePairs[i].endswith('LA'):
        binanceQuoteIDs[binanceUniquePairs[i]] = '707ccdf1-af98-4e09-95fc-e685ed0ae4c6'
        binanceQuoteTag[binanceUniquePairs[i]] = 'LA'
        binanceCurTag[binanceUniquePairs[i]] = binanceUniquePairs[i][:-2]

binance_trades['quote'] = binance_trades.pair.map(binanceQuoteIDs)
binance_trades['quote1'] = binance_trades.pair.map(binanceQuoteTag)
binance_trades['currency1'] = binance_trades.pair.map(binanceCurTag)
binance_trades['currency'] = binance_trades.currency1.map(currencies_ids)

# This section: Joining LATOKEN and Binance trades and adjusting data in the final table
# Joining with Binance trades
trades = pd.concat([latoken_trades,binance_trades], ignore_index=True)

# Making all trades directions correct (if we were takers, then the direction is opposite, because direction is set by maker)
# Adjusting signs (quantity is negative is we buy, cost is negative if we sell)
for i in range(len(trades)):
    if trades.status.iloc[i]=='taker':
        if trades.direction.iloc[i]=='BUY':
            trades.direction.iloc[i]='SELL'
        else:
            trades.direction.iloc[i]='BUY'
#trades['direction'] = trades[trades.status=='taker'].direction.replace(['BUY', 'SELL'],['SELL', 'BUY'])
trades.loc[trades[trades.direction=='BUY'].index, 'cost'] *= -1
trades.loc[trades[trades.direction=='SELL'].index, 'quantity'] *= -1

# Adding two required columns to latoken_trades
trades['account'] = trades.trader.map(account_type)

# This section: Getting opening and closing prices
# Creating list of unique tokens for iterations, opening prices dictionary, closing prices dictionary, bin list
unique_currencies = list(set(trades.quote1.unique())|set(trades.currency1.unique()))
currency_prices_last = dict()
currency_prices_current = dict()
unknown = list()

# Timestamps for Binance queries (Binance server uses UTC time)
start = int(str(int(start_date.replace(tzinfo=datetime.timezone.utc).timestamp()))+'000')
end = int(str(int(end_date.replace(tzinfo=datetime.timezone.utc).timestamp()))+'000')

# Getting active currencies
active = active_currencies(tuple(trades.currency.dropna().unique())).drop_duplicates()
active = active.drop(active[(active.currency.isin(active[active.row==2].currency.unique()))&(active.row==1)].index)
active['currency'] = active.currency.map(latoken_currencies)
active.index = active.currency
active = active.drop(columns=['currency'])
active = active.to_dict()['status']

# Getting period opening and closing prices
for i in range(len(unique_currencies)):
    if unique_currencies[i]=='USDT':
        currency_prices_current[unique_currencies[i]] = 1
        currency_prices_last[unique_currencies[i]] = 1
    else:
        # Collecting prices from Binance
        try:
            # We check if the list is returned, then collect data. Errors are returned as dictionaries.
            if type(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}USDT&interval=1h&startTime={start}&endTime={end}").json())==list:
                currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}USDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
                currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}USDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
            elif type(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}BTC&interval=1h&startTime={start}&endTime={end}").json())==list:
                currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}BTC&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
                currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}BTC&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
            elif type(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}ETH&interval=1h&startTime={start}&endTime={end}").json())==list:
                currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}ETH&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
                currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}ETH&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
            # This is mainly for BCHA token, it is very rare for a token to be traded on Binance for BUSD only
            else:
                currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}BUSD&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=BUSDUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
                currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={unique_currencies[i]}BUSD&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=BUSDUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
        # Tokens that are not traded on Binance are illiquid, so we get the latest LATOKEN price (for closing period prices)
        # and we get also get the current LATOKEN prices (for opening period prices) - this can be changed (Coingecko prices can be taken as tickets coinside, LATOKEN API is limited to a week and doesn't have timestamps so hard to identify the period of a price returned)
        except:
            try:
                if active[unique_currencies[i]] == 'DISABLED':
                    currency_prices_current[unique_currencies[i]] = 0
                    currency_prices_last[unique_currencies[i]] = 0
                    unknown.append(unique_currencies[i])
                else:
                    try:
                        if requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/USDT").json()['lastPrice']!='0':
                            currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/USDT").json()['lastPrice'])
                            currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/USDT").json()['lastPrice'])
                        # Sometimes there are no x/USDT pair on LATOKEN for this pair, so we look at x/BTC and x/ETH current prices
                        else:
                            if requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/BTC").json()['lastPrice']!='0':
                                currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/BTC").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/BTC/USDT").json()['lastPrice'])
                                currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/BTC").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/BTC/USDT").json()['lastPrice'])
                            else:
                                if requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/ETH").json()['lastPrice']!='0':
                                    currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/ETH").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/ETH/USDT").json()['lastPrice'])
                                    currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/ETH").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/ETH/USDT").json()['lastPrice'])
                                # If there are no prices on LATOKEN, then we set the price equal to zero, we can check all these tokens in the 'unknown' list
                                else: 
                                    currency_prices_current[unique_currencies[i]] = 0
                                    currency_prices_last[unique_currencies[i]] = 0
                                    unknown.append(unique_currencies[i])
                    # This exception is for tokens that are not traded neither on Binance nor on LATOKEN. 
                    except:
                        currency_prices_current[unique_currencies[i]] = 0
                        currency_prices_last[unique_currencies[i]] = 0
                        unknown.append(unique_currencies[i])
            except:
                try:
                    if requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/USDT").json()['lastPrice']!='0':
                        currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/USDT").json()['lastPrice'])
                        currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/USDT").json()['lastPrice'])
                    # Sometimes there are no x/USDT pair on LATOKEN for this pair, so we look at x/BTC and x/ETH current prices
                    else:
                        if requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/BTC").json()['lastPrice']!='0':
                            currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/BTC").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/BTC/USDT").json()['lastPrice'])
                            currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/BTC").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/BTC/USDT").json()['lastPrice'])
                        else:
                            if requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/ETH").json()['lastPrice']!='0':
                                currency_prices_current[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/ETH").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/ETH/USDT").json()['lastPrice'])
                                currency_prices_last[unique_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{unique_currencies[i]}/ETH").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/ETH/USDT").json()['lastPrice'])
                            # If there are no prices on LATOKEN, then we set the price equal to zero, we can check all these tokens in the 'unknown' list
                            else: 
                                currency_prices_current[unique_currencies[i]] = 0
                                currency_prices_last[unique_currencies[i]] = 0
                                unknown.append(unique_currencies[i])
                # This exception is for tokens that are not traded neither on Binance nor on LATOKEN. 
                except:
                    currency_prices_current[unique_currencies[i]] = 0
                    currency_prices_last[unique_currencies[i]] = 0
                    unknown.append(unique_currencies[i])

                
### Updating opening prices with closing prices of previous period
if mode == 'daily':
    updating = pd.read_csv(f'{os.getcwd()}/prices_daily.csv')
    updating.index = updating['Unnamed: 0']
    updating = updating.drop(columns=['Unnamed: 0'])
    updating = updating.to_dict()['0']
    currency_prices_last.update(updating)
elif mode == 'weekly':
    updating = pd.read_csv(f'{os.getcwd()}/prices_weekly.csv')
    updating.index = updating['Unnamed: 0']
    updating = updating.drop(columns=['Unnamed: 0'])
    updating = updating.to_dict()['0']
    currency_prices_last.update(updating)
elif mode == 'monthly':
    updating = pd.read_csv(f'{os.getcwd()}/prices_monthly.csv')
    updating.index = updating['Unnamed: 0']
    updating = updating.drop(columns=['Unnamed: 0'])
    updating = updating.to_dict()['0']
    currency_prices_last.update(updating)
elif mode == 'retro':
    updating = pd.read_csv(f'{os.getcwd()}/prices_retro.csv')
    updating.index = updating['Unnamed: 0']
    updating = updating.drop(columns=['Unnamed: 0'])
    updating = updating.to_dict()['0']
    currency_prices_last.update(updating)
else:
    pass
                
# Adding volume column to trades table
trades['volume'] = abs(trades.quote1.map(currency_prices_current) * trades.cost)

# General Account
# This section: Creating a PnL final analysis table
pnl_analysisGeneral = trades[trades.account.isin(['65127','leverage','binance_trading','margin'])].groupby(['pair', 'currency1', 'quote1'])[['quantity', 'cost']].sum()
pnl_analysisGeneral = pnl_analysisGeneral.reset_index()
pnl_analysisGeneral['quantityUSD'] = pnl_analysisGeneral.currency1.map(currency_prices_current) * pnl_analysisGeneral.quantity
pnl_analysisGeneral['costUSD'] = pnl_analysisGeneral.quote1.map(currency_prices_current) * pnl_analysisGeneral.cost
pnl_analysisGeneral['pnl'] = pnl_analysisGeneral.quantityUSD + pnl_analysisGeneral.costUSD
pnl_analysisGeneral = pnl_analysisGeneral[['pair', 'quantity', 'cost', 'quantityUSD', 'costUSD', 'pnl']]
pnl_analysisGeneral = pnl_analysisGeneral.round({'quantity': 2, 'cost': 2, 'quantityUSD': 2, 'costUSD': 2, 'pnl': 2})
pnl_analysisGeneral = pnl_analysisGeneral.sort_values(by='pnl', axis=0, ascending=True)

# Checking for outliers
pnl_outliersGeneral = pnl_analysisGeneral[(pnl_analysisGeneral.pnl>100)|(pnl_analysisGeneral.pnl<-100)]
pnl_outliersGeneral = pnl_outliersGeneral.sort_values(by='pnl', axis=0, ascending=True)


# Floating bot account
# This section: Creating a PnL final analysis table
pnl_analysisFloating = trades[trades.account.isin(['floating'])].groupby(['pair', 'currency1', 'quote1'])[['quantity', 'cost']].sum()
pnl_analysisFloating = pnl_analysisFloating.reset_index()
pnl_analysisFloating['quantityUSD'] = pnl_analysisFloating.currency1.map(currency_prices_current) * pnl_analysisFloating.quantity
pnl_analysisFloating['costUSD'] = pnl_analysisFloating.quote1.map(currency_prices_current) * pnl_analysisFloating.cost
pnl_analysisFloating['pnl'] = pnl_analysisFloating.quantityUSD + pnl_analysisFloating.costUSD
pnl_analysisFloating = pnl_analysisFloating[['pair', 'quantity', 'cost', 'quantityUSD', 'costUSD', 'pnl']]
pnl_analysisFloating = pnl_analysisFloating.round({'quantity': 2, 'cost': 2, 'quantityUSD': 2, 'costUSD': 2, 'pnl': 2})
pnl_analysisFloating = pnl_analysisFloating.sort_values(by='pnl', axis=0, ascending=True)

# Checking for outliers
pnl_outliersFloating = pnl_analysisFloating[(pnl_analysisFloating.pnl>30)|(pnl_analysisFloating.pnl<-30)]
pnl_outliersFloating = pnl_outliersFloating.sort_values(by='pnl', axis=0, ascending=True)


# Rotation bot account
# This section: Creating a PnL final analysis table
pnl_analysisRotation = trades[trades.account.isin(['rotation'])].groupby(['pair', 'currency1', 'quote1'])[['quantity', 'cost']].sum()
pnl_analysisRotation = pnl_analysisRotation.reset_index()
pnl_analysisRotation['quantityUSD'] = pnl_analysisRotation.currency1.map(currency_prices_current) * pnl_analysisRotation.quantity
pnl_analysisRotation['costUSD'] = pnl_analysisRotation.quote1.map(currency_prices_current) * pnl_analysisRotation.cost
pnl_analysisRotation['pnl'] = pnl_analysisRotation.quantityUSD + pnl_analysisRotation.costUSD
pnl_analysisRotation = pnl_analysisRotation[['pair', 'quantity', 'cost', 'quantityUSD', 'costUSD', 'pnl']]
pnl_analysisRotation = pnl_analysisRotation.round({'quantity': 2, 'cost': 2, 'quantityUSD': 2, 'costUSD': 2, 'pnl': 2})
pnl_analysisRotation = pnl_analysisRotation.sort_values(by='pnl', axis=0, ascending=True)

# Checking for outliers
pnl_outliersRotation = pnl_analysisRotation[(pnl_analysisRotation.pnl>50)|(pnl_analysisRotation.pnl<-50)]
pnl_outliersRotation = pnl_outliersRotation.sort_values(by='pnl', axis=0, ascending=True)


# LaLiquidity account
# This section: Creating a PnL final analysis table
pnl_analysisLiquidity = trades[trades.account.isin(['laliquidity'])].groupby(['pair', 'currency1', 'quote1'])[['quantity', 'cost']].sum()
pnl_analysisLiquidity = pnl_analysisLiquidity.reset_index()
pnl_analysisLiquidity['quantityUSD'] = pnl_analysisLiquidity.currency1.map(currency_prices_current) * pnl_analysisLiquidity.quantity
pnl_analysisLiquidity['costUSD'] = pnl_analysisLiquidity.quote1.map(currency_prices_current) * pnl_analysisLiquidity.cost
pnl_analysisLiquidity['pnl'] = pnl_analysisLiquidity.quantityUSD + pnl_analysisLiquidity.costUSD
pnl_analysisLiquidity = pnl_analysisLiquidity[['pair', 'quantity', 'cost', 'quantityUSD', 'costUSD', 'pnl']]
pnl_analysisLiquidity = pnl_analysisLiquidity.round({'quantity': 2, 'cost': 2, 'quantityUSD': 2, 'costUSD': 2, 'pnl': 2})
pnl_analysisLiquidity = pnl_analysisLiquidity.sort_values(by='pnl', axis=0, ascending=True)

# Checking for outliers
pnl_outliersLiquidity = pnl_analysisLiquidity[(pnl_analysisLiquidity.pnl>50)|(pnl_analysisLiquidity.pnl<-50)]
pnl_outliersLiquidity = pnl_outliersLiquidity.sort_values(by='pnl', axis=0, ascending=True)

In [25]:
# Checkers
# pnl_analysis[pnl_analysis.pnl.isna()]
# trades[trades.pair=='BNBUSDT']
# unknown

In [26]:
# This section: Calculates Binance fees
# Getting Binance fees data
if binance_mode == 'db':
    fees_paid_binance = binance_fees(start_date, end_date)
    fees_paid_binance = fees_paid_binance.rename({0:'commission_asset', 1: 'fee_paid_USD'}, axis=1)
elif binance_mode == 'csv':
    fees_paid_binance = pd.DataFrame(binanceFees).reset_index()
    fees_paid_binance = fees_paid_binance.rename({'fee_asset':'commission_asset', 'Fee': 'fee_paid_USD'}, axis=1)

# Checking that all assets that we paid fees in have a price in a dictionary (usually BNB doesn't have), otherwise add
for i in range(len(fees_paid_binance)):
    if fees_paid_binance.commission_asset.iloc[i] not in currency_prices_current.keys():
        asset = fees_paid_binance.commission_asset.iloc[i]
        request_binance = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={asset}USDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
        currency_prices_current.update({asset: request_binance})
        request_binance = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={asset}USDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
        currency_prices_last.update({asset: request_binance})

# Converting fees from currency they were paid in to USD and formatting
fees_paid_binance['fee_paid_USD'] = fees_paid_binance.commission_asset.map(currency_prices_current) * fees_paid_binance.fee_paid_USD
total_binance_fees = fees_paid_binance.fee_paid_USD.sum().round(2)
fees_paid_binance['fee_paid_USD'] = fees_paid_binance['fee_paid_USD'].round(2)

# Adding 'total' rows to the dataframe and styling  
total_latoken_volume = trades[trades.account.isin(['65127','leverage','margin'])].volume.sum().round(2)
total_binance_volume = trades[trades.account.isin(['binance_trading'])].volume.sum().round(2)
total_dataframe = pd.DataFrame({'commission_asset':[' ', 'Binance fees:', 'LATOKEN volume:', 'Binance volume:'],
                                             'fee_paid_USD':[' ', f'{"{:,}".format(total_binance_fees)}', 
                                                             f'{"{:,}".format(total_latoken_volume)}', 
                                                             f'{"{:,}".format(total_binance_volume)}']})
fees_paid_binance = fees_paid_binance.append(total_dataframe, ignore_index=True)

# fees_paid_binance

In [27]:
asd = 0
asdq = 0
# This section: Getting balances & transactions
first = datetime.datetime.now()
# Opening and closing balances
balances_opening = balances_current(start_date)
balances_closing = balances_current(end_date)
if asd == 1:
    qwe = pd.read_excel('/Users/macbook_nikita/Desktop/qwe.xlsx', sheet_name = 'exposureFloating')
    qwer = pd.read_csv(f'{os.getcwd()}/prices_monthly.csv')
    qwer.index = qwer['Unnamed: 0']
    qwer = qwer.drop(columns=['Unnamed: 0'])
    qwer = qwer.to_dict()['0']
    qwert = qwe[['closing, $', 'Unnamed: 0']]
    qwert['prices'] = qwert['Unnamed: 0'].map(qwer)
    qwert = qwert.drop([0], axis=0)
    qwert = qwert[qwert['closing, $']!=0]
    qwert['balance'] = qwert['closing, $']/qwert['prices']
    qwert = qwert[['Unnamed: 0', 'balance']].rename({'Unnamed: 0': 'currency', 'balance': 'total'}, axis=1)
    qwert['investor_id'] = 'FLOATINGBOT'
    startBalanceFloating = qwert.copy(deep=True)
else:
    startBalanceFloating = balances_current_floating(start_date)

if asdq == 1:
    qwe = pd.read_excel('/Users/macbook_nikita/Desktop/qwe2.xlsx', sheet_name = 'exposureFloating')
    qwer = pd.read_csv(f'{os.getcwd()}/prices_weekly.csv')
    qwer.index = qwer['Unnamed: 0']
    qwer = qwer.drop(columns=['Unnamed: 0'])
    qwer = qwer.to_dict()['0']
    qwert = qwe[['closing, $', 'Unnamed: 0']]
    qwert['prices'] = qwert['Unnamed: 0'].map(qwer)
    qwert = qwert.drop([0], axis=0)
    qwert = qwert[qwert['closing, $']!=0]
    qwert['balance'] = qwert['closing, $']/qwert['prices']
    qwert = qwert[['Unnamed: 0', 'balance']].rename({'Unnamed: 0': 'currency', 'balance': 'total'}, axis=1)
    qwert['investor_id'] = 'FLOATINGBOT'
    endBalanceFloating = qwert.copy(deep=True)
else:
    endBalanceFloating = balances_current_floating(end_date)
second = datetime.datetime.now()
print(f'balances received in {second-first}')

# Getting transfers on LATOKEN
latoken_transfers = transfers_latoken(start_date, end_date)

# Getting transactions on LATOKEN
latoken_transactions = transactions_latoken(start_date, end_date)
latoken_transactions.loc[latoken_transactions[latoken_transactions.transaction_type=='WITHDRAWAL'].index, 'value'] *= -1

# Getting adjustments (if adjustments happened during the analysed week, they are called new and are only subtracted from the closing balance)
adj_transactions = transaction_adjustments()
adj_transactions_new = adj_transactions[adj_transactions.updated>=start_date]
adj_transactions_old = adj_transactions[adj_transactions.updated<start_date]
adj_transfer = transfer_adjustments()
adj_transfer_new = adj_transfer[adj_transfer.updated>=start_date]
adj_transfer_old = adj_transfer[adj_transfer.updated<start_date]

otherAccountsAdjustments = transfer_adjustments_other()
otherAccountsAdjustments['account'] = otherAccountsAdjustments.account.map(account_type)

# Transactions on Binance
transactions_binance = binance_transactions(start_date, end_date).drop_duplicates()
transactions_binance.loc[transactions_binance[transactions_binance.type=='WITHDRAWAL'].index, 'amount'] *= -1
third = datetime.datetime.now()
print(f'transactions and transfers and adjustments received in {third-second}')

# This section: Wrangling with currency ids and prices
# Checking for new currencies and changing ids to tickers
all_currencies = list(((((set(balances_opening.currency)|set(balances_closing.currency))|
                        (set(latoken_transactions.currency_tag)|set(latoken_transfers.from_asset_tag)))|
                       (set(adj_transfer.currency_tag)|set(adj_transactions.currency_tag)))|
                      (set(transactions_binance.currency)|set(otherAccountsAdjustments.currency_tag)))|
                        (set(endBalanceFloating.currency)|set(startBalanceFloating.currency)))
new_currencies = [currency for currency in all_currencies if len(currency)>10]
if len(new_currencies) != 0:
    new_currencies_tuple = tuple(new_currencies)
    new_currencies = currencies(new_currencies_tuple)
    new_currencies.index = new_currencies.id
    new_currencies = new_currencies.drop(columns=['id'])
    new_currencies = new_currencies.to_dict()['tag']
    all_currencies = [new_currencies[currency] if currency in new_currencies.keys() else currency for currency in all_currencies]

def cur_change(x):
    if len(x)>10:
        x = new_currencies[x]
    return x

balances_closing['currency'] = balances_closing.currency.map(cur_change)
balances_opening['currency'] = balances_opening.currency.map(cur_change)
latoken_transactions['currency_tag'] = latoken_transactions.currency_tag.map(cur_change)
latoken_transfers['from_asset_tag'] = latoken_transfers.from_asset_tag.map(cur_change)

# Getting active currencies
all_ids = ids(tuple(all_currencies))

all_active = active_currencies(tuple(all_ids.id.dropna().unique())).drop_duplicates()
all_active = all_active.drop(all_active[(all_active.currency.isin(all_active[all_active.row==2].currency.unique()))&(all_active.row==1)].index)
all_active.index = all_active.currency
all_active = all_active.drop(columns=['currency', 'row'])
all_active = all_active.to_dict()['status']

all_ids['status'] = all_ids.id.map(all_active)
all_ids.index = all_ids.tag
all_ids = all_ids.drop(columns=['tag', 'id'])
all_ids = all_ids.to_dict()['status']
fourth = datetime.datetime.now()
print(f'currencies are dealt with in {fourth-third}')

# Checking for all prices to be in the dictionary and converting fees in USD
for i in range(len(all_currencies)):
    if all_currencies[i] not in currency_prices_current.keys():
        # Collecting prices from Binance. Binance prices are in priority comparing to LATOKEN prices.
        try:
            # We check if the list is returned, then collect data. Errors are returned as dictionaries.
            if type(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}USDT&interval=1h&startTime={start}&endTime={end}&limit=999").json())==list:
                currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}USDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
                currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}USDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
            elif type(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}BTC&interval=1h&startTime={start}&endTime={end}&limit=999").json())==list:
                currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}BTC&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
                currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}BTC&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
            elif type(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}ETH&interval=1h&startTime={start}&endTime={end}&limit=999").json())==list:
                currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}ETH&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
                currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}ETH&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
            # This is mainly for BCHA token, it is very rare for a token to be traded on Binance for BUSD only
            else:
                currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}BUSD&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=BUSDUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[-1][4])
                currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.binance.com/api/v3/klines?symbol={all_currencies[i]}BUSD&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4]) * float(requests.get(f"https://api.binance.com/api/v3/klines?symbol=BUSDUSDT&interval=1h&startTime={start}&endTime={end}&limit=999").json()[0][4])
        # Tokens that are not traded on Binance are illiquid, so we get the latest LATOKEN price (for closing period prices)
        # and we get also get the current LATOKEN prices (for opening period prices) - this can be changed (Coingecko prices can be taken as tickets coinside, LATOKEN API is limited to a week and doesn't have timestamps so hard to identify the period of a price returned)
        except:
            try:
                if all_ids[all_currencies[i]] == 'DISABLED':
                    currency_prices_current[all_currencies[i]] = 0
                    currency_prices_last[all_currencies[i]] = 0
                    unknown.append(all_currencies[i])
                else:
                    try:
                        if requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/USDT").json()['lastPrice']!='0':
                            currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/USDT").json()['lastPrice'])
                            currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/USDT").json()['lastPrice'])
                        # Sometimes there are no x/USDT pair on LATOKEN for this pair, so we look at x/BTC and x/ETH current prices
                        else:
                            if requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/BTC").json()['lastPrice']!='0':
                                currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/BTC").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/BTC/USDT").json()['lastPrice'])
                                currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/BTC").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/BTC/USDT").json()['lastPrice'])
                            else:
                                if requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/ETH").json()['lastPrice']!='0':
                                    currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/ETH").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/ETH/USDT").json()['lastPrice'])
                                    currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/ETH").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/ETH/USDT").json()['lastPrice'])
                                # If there are no prices on LATOKEN, then we set the price equal to zero, we can check all these tokens in the 'unknown' list
                                else: 
                                    currency_prices_current[all_currencies[i]] = 0
                                    currency_prices_last[all_currencies[i]] = 0
                                    unknown.append(all_currencies[i])
                    # This exception is for tokens that are not traded neither on Binance nor on LATOKEN. 
                    except:
                        currency_prices_current[all_currencies[i]] = 0
                        currency_prices_last[all_currencies[i]] = 0
                        unknown.append(all_currencies[i])
            except:
                try:
                    if requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/USDT").json()['lastPrice']!='0':
                        currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/USDT").json()['lastPrice'])
                        currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/USDT").json()['lastPrice'])
                    # Sometimes there are no x/USDT pair on LATOKEN for this pair, so we look at x/BTC and x/ETH current prices
                    else:
                        if requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/BTC").json()['lastPrice']!='0':
                            currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/BTC").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/BTC/USDT").json()['lastPrice'])
                            currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/BTC").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/BTC/USDT").json()['lastPrice'])
                        else:
                            if requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/ETH").json()['lastPrice']!='0':
                                currency_prices_current[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/ETH").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/ETH/USDT").json()['lastPrice'])
                                currency_prices_last[all_currencies[i]] = float(requests.get(f"https://api.latoken.com/v2/ticker/{all_currencies[i]}/ETH").json()['lastPrice']) * float(requests.get(f"https://api.latoken.com/v2/ticker/ETH/USDT").json()['lastPrice'])
                            # If there are no prices on LATOKEN, then we set the price equal to zero, we can check all these tokens in the 'unknown' list
                            else: 
                                currency_prices_current[all_currencies[i]] = 0
                                currency_prices_last[all_currencies[i]] = 0
                                unknown.append(all_currencies[i])
                # This exception is for tokens that are not traded neither on Binance nor on LATOKEN. 
                except:
                    currency_prices_current[all_currencies[i]] = 0
                    currency_prices_last[all_currencies[i]] = 0
                    unknown.append(all_currencies[i])
fifth = datetime.datetime.now()
print(f'prices received in {fifth-fourth}')
### Updating opening prices with closing prices of previous period
if (mode == 'daily') or (mode == 'weekly') or (mode == 'monthly'):
    currency_prices_last.update(updating)
else:
    pass

# Getting value of deposit and withdrawal fees on Binance
transactions_binance['fee'] = transactions_binance.currency.map(currency_prices_current) * transactions_binance.fee

# This section: Creating 4 tables that go into Total table that is pivoted into Exposure table
# Internal_mm_other tab
internal_mm_other = pd.DataFrame(index = (set(balances_opening[balances_opening.exchange=='BINANCE'].currency.unique())|set(balances_closing[balances_closing.exchange=='BINANCE'].currency.unique()))|(set(binance_trades.currency1.unique())|set(transactions_binance.currency.unique())))
internal_mm_other = internal_mm_other.join(balances_closing[balances_closing.exchange=='BINANCE'].groupby(['currency'])['total'].sum()).rename({'total': 'closing'}, axis=1).fillna(0)
internal_mm_other = internal_mm_other.join(balances_opening[balances_opening.exchange=='BINANCE'].groupby(['currency'])['total'].sum()).rename({'total': 'opening'}, axis=1).fillna(0)
internal_mm_other = internal_mm_other.join(trades[(trades.account=='binance_trading')&(trades.quantity.values>0)].groupby(['currency1'])['quantity'].sum()).rename({'quantity': 'trade_buy_q'}, axis=1).fillna(0)
internal_mm_other = internal_mm_other.join(trades[(trades.account=='binance_trading')&(trades.cost.values>0)].groupby(['quote1'])['cost'].sum()).rename({'cost': 'trade_buy_c'}, axis=1).fillna(0)
internal_mm_other = internal_mm_other.join(trades[(trades.account=='binance_trading')&(trades.quantity.values<0)].groupby(['currency1'])['quantity'].sum()).fillna(0)
internal_mm_other = internal_mm_other.join(trades[(trades.account=='binance_trading')&(trades.cost.values<0)].groupby(['quote1'])['cost'].sum()).fillna(0)
internal_mm_other['transfers_in, $'] = 0
internal_mm_other['transfers_out, $'] = 0
internal_mm_other = internal_mm_other.join(transactions_binance[transactions_binance.type=='Deposit'].groupby(['currency'])['amount'].sum()).rename({'amount': 'deposits'}, axis=1).fillna(0)
internal_mm_other = internal_mm_other.join(transactions_binance[transactions_binance.type=='Withdraw'].groupby(['currency'])['amount'].sum()).rename({'amount': 'withdrawals'}, axis=1).fillna(0)
internal_mm_other['trade_buy'] = internal_mm_other.trade_buy_q + internal_mm_other.trade_buy_c
internal_mm_other['trade_sell'] = internal_mm_other.quantity + internal_mm_other.cost
internal_mm_other = internal_mm_other.drop(columns=['quantity', 'cost', 'trade_buy_q', 'trade_buy_c'])
internal_mm_other['rate_current'] = internal_mm_other.index.map(currency_prices_current)
internal_mm_other['tier'] = internal_mm_other.index.map(token_tier).fillna('tier 3')
internal_mm_other['unexplained_currency'] = internal_mm_other.closing - (internal_mm_other.opening + internal_mm_other.trade_buy + internal_mm_other.trade_sell + internal_mm_other.deposits + internal_mm_other.withdrawals)
internal_mm_other['unexplained_usd'] = internal_mm_other.unexplained_currency * internal_mm_other.rate_current
internal_mm_other['adj_closing'] = internal_mm_other.closing
internal_mm_other['adj_opening'] = internal_mm_other.opening
internal_mm_other['opening, $'] = internal_mm_other.opening * internal_mm_other.rate_current
internal_mm_other['closing, $'] = internal_mm_other.closing * internal_mm_other.rate_current
internal_mm_other['adj_closing, $'] = internal_mm_other.adj_closing * internal_mm_other.rate_current
internal_mm_other['adj_opening, $'] = internal_mm_other.adj_opening * internal_mm_other.rate_current
internal_mm_other['trade_buy, $'] = internal_mm_other.trade_buy * internal_mm_other.rate_current
internal_mm_other['trade_sell, $'] = internal_mm_other.trade_sell * internal_mm_other.rate_current
internal_mm_other['deposits, $'] = internal_mm_other.deposits * internal_mm_other.rate_current
internal_mm_other['withdrawals, $'] = internal_mm_other.withdrawals * internal_mm_other.rate_current
internal_mm_other['wallet_id'] = 'binance_trading'
internal_mm_other = internal_mm_other[['opening', 'adj_opening', 'opening, $', 'adj_opening, $', 'trade_buy, $', 'trade_sell, $',
                     'deposits, $', 'withdrawals, $','transfers_in, $', 'transfers_out, $', 'closing', 
                     'adj_closing', 'closing, $', 'adj_closing, $', 'unexplained_currency', 'unexplained_usd',
                     'rate_current', 'tier', 'wallet_id']]

sixth = datetime.datetime.now()
print(f'immo done in {sixth-fifth}')

# Marginal_acc 
marginal_acc = pd.DataFrame(index = (set(balances_opening[balances_opening.investor_id=='888888'].currency.unique())|set(balances_closing[balances_closing.investor_id=='888888'].currency.unique()))|(set(adj_transfer.currency_tag)|set(adj_transactions.currency_tag)))
marginal_acc = marginal_acc.join(balances_closing[balances_closing.investor_id=='888888'].groupby(['currency'])['total'].sum()).rename({'total': 'closing'}, axis=1).fillna(0)
marginal_acc = marginal_acc.join(balances_opening[balances_opening.investor_id=='888888'].groupby(['currency'])['total'].sum()).rename({'total': 'opening'}, axis=1).fillna(0)
if adj_transfer_new.empty and adj_transactions_new.empty:
    adjustments = pd.concat([adj_transactions, adj_transfer])
    adjustments = adjustments.groupby(['currency_tag'])['amount'].sum()
    marginal_acc = marginal_acc.join(adjustments).rename({'amount': 'adjustments'}, axis=1).fillna(0)
    marginal_acc['adj_closing'] = marginal_acc.closing - marginal_acc.adjustments
    marginal_acc['adj_opening'] = marginal_acc.opening - marginal_acc.adjustments
elif adj_transfer_new.empty==False or adj_transactions_new.empty==False:
    adjustments = pd.concat([adj_transactions_old, adj_transfer_old])
    adjustments = adjustments.groupby(['currency_tag'])['amount'].sum()
    adjustments_new = pd.concat([adj_transactions_new, adj_transfer_new])
    adjustments_new = adjustments_new.groupby(['currency_tag'])['amount'].sum()
    marginal_acc = marginal_acc.join(adjustments).rename({'amount': 'adjustments'}, axis=1).fillna(0)
    marginal_acc = marginal_acc.join(adjustments_new).rename({'amount': 'adjustments_new'}, axis=1).fillna(0)
    marginal_acc['adj_closing'] = marginal_acc.closing - marginal_acc.adjustments - marginal_acc.adjustments_new
    marginal_acc['adj_opening'] = marginal_acc.opening - marginal_acc.adjustments
marginal_acc = marginal_acc.join(trades[(trades.account=='margin')&(trades.quantity.values>0)].groupby(['currency1'])['quantity'].sum()).rename({'quantity': 'trade_buy_q'}, axis=1).fillna(0)
marginal_acc = marginal_acc.join(trades[(trades.account=='margin')&(trades.cost.values>0)].groupby(['quote1'])['cost'].sum()).rename({'cost': 'trade_buy_c'}, axis=1).fillna(0)
marginal_acc = marginal_acc.join(trades[(trades.account=='margin')&(trades.quantity.values<0)].groupby(['currency1'])['quantity'].sum()).fillna(0)
marginal_acc = marginal_acc.join(trades[(trades.account=='margin')&(trades.cost.values<0)].groupby(['quote1'])['cost'].sum()).fillna(0)
marginal_acc = marginal_acc.join(latoken_transfers[latoken_transfers.to_user_id=='ddee6a46-3bbd-4d39-b561-c028e618d8ff'].groupby(['from_asset_tag'])['amount'].sum()).rename({'amount': 'transfers_in'}, axis=1).fillna(0)
marginal_acc = marginal_acc.join(latoken_transfers[latoken_transfers.from_user_id=='ddee6a46-3bbd-4d39-b561-c028e618d8ff'].groupby(['from_asset_tag'])['amount'].sum() * (-1)).rename({'amount': 'transfers_out'}, axis=1).fillna(0)
marginal_acc = marginal_acc.join(latoken_transactions[(latoken_transactions.user_id=='ddee6a46-3bbd-4d39-b561-c028e618d8ff')&(latoken_transactions.transaction_type=='DEPOSIT')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'deposits'}, axis=1).fillna(0)
marginal_acc = marginal_acc.join(latoken_transactions[(latoken_transactions.user_id=='ddee6a46-3bbd-4d39-b561-c028e618d8ff')&(latoken_transactions.transaction_type=='WITHDRAWAL')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'withdrawals'}, axis=1).fillna(0)
marginal_acc['trade_buy'] = marginal_acc.trade_buy_q + marginal_acc.trade_buy_c
marginal_acc['trade_sell'] = marginal_acc.quantity + marginal_acc.cost
marginal_acc['rate_current'] = marginal_acc.index.map(currency_prices_current)
marginal_acc['tier'] = marginal_acc.index.map(token_tier).fillna('tier 3')
marginal_acc = marginal_acc.drop(columns=['adjustments', 'quantity', 'cost', 'trade_buy_q', 'trade_buy_c'])
marginal_acc['unexplained_currency'] = marginal_acc.closing - (marginal_acc.opening + marginal_acc.transfers_in +
                                                              marginal_acc.transfers_out + marginal_acc.deposits +
                                                              marginal_acc.withdrawals + marginal_acc.trade_buy +
                                                              marginal_acc.trade_sell)
marginal_acc['unexplained_usd'] = marginal_acc.unexplained_currency * marginal_acc.rate_current
marginal_acc['opening, $'] = marginal_acc.opening * marginal_acc.rate_current
marginal_acc['closing, $'] = marginal_acc.closing * marginal_acc.rate_current
marginal_acc['adj_closing, $'] = marginal_acc.adj_closing * marginal_acc.rate_current
marginal_acc['adj_opening, $'] = marginal_acc.adj_opening * marginal_acc.rate_current
marginal_acc['trade_buy, $'] = marginal_acc.trade_buy * marginal_acc.rate_current
marginal_acc['trade_sell, $'] = marginal_acc.trade_sell * marginal_acc.rate_current
marginal_acc['deposits, $'] = marginal_acc.deposits * marginal_acc.rate_current
marginal_acc['withdrawals, $'] = marginal_acc.withdrawals * marginal_acc.rate_current
marginal_acc['transfers_in, $'] = marginal_acc.transfers_in * marginal_acc.rate_current
marginal_acc['transfers_out, $'] = marginal_acc.transfers_out * marginal_acc.rate_current
marginal_acc['wallet_id'] = '692129'
marginal_acc = marginal_acc[['opening', 'adj_opening', 'opening, $', 'adj_opening, $', 'trade_buy, $', 'trade_sell, $',
                 'deposits, $', 'withdrawals, $','transfers_in, $', 'transfers_out, $', 'closing', 
                 'adj_closing', 'closing, $', 'adj_closing, $', 'unexplained_currency', 'unexplained_usd',
                 'rate_current', 'tier', 'wallet_id']]

seventh = datetime.datetime.now()
print(f'mar done in {seventh-sixth}')

# Leverage_acc
leverage_acc = pd.DataFrame(index = set(balances_opening[balances_opening.investor_id=='693752'].currency.unique())|set(balances_closing[balances_closing.investor_id=='693752'].currency.unique()))
leverage_acc = leverage_acc.join(balances_closing[balances_closing.investor_id=='693752'].groupby(['currency'])['total'].sum()).rename({'total': 'closing'}, axis=1).fillna(0)
leverage_acc = leverage_acc.join(balances_opening[balances_opening.investor_id=='693752'].groupby(['currency'])['total'].sum()).rename({'total': 'opening'}, axis=1).fillna(0)
leverage_acc = leverage_acc.join(trades[(trades.account=='leverage')&(trades.quantity.values>0)].groupby(['currency1'])['quantity'].sum()).rename({'quantity': 'trade_buy_q'}, axis=1).fillna(0)
leverage_acc = leverage_acc.join(trades[(trades.account=='leverage')&(trades.cost.values>0)].groupby(['quote1'])['cost'].sum()).rename({'cost': 'trade_buy_c'}, axis=1).fillna(0)
leverage_acc = leverage_acc.join(trades[(trades.account=='leverage')&(trades.quantity.values<0)].groupby(['currency1'])['quantity'].sum()).fillna(0)
leverage_acc = leverage_acc.join(trades[(trades.account=='leverage')&(trades.cost.values<0)].groupby(['quote1'])['cost'].sum()).fillna(0)
leverage_acc = leverage_acc.join(latoken_transfers[latoken_transfers.to_user_id=='8fdd328f-47cc-4705-bf45-9a4f20314b87'].groupby(['from_asset_tag'])['amount'].sum()).rename({'amount': 'transfers_in'}, axis=1).fillna(0)
leverage_acc = leverage_acc.join(latoken_transfers[latoken_transfers.from_user_id=='8fdd328f-47cc-4705-bf45-9a4f20314b87'].groupby(['from_asset_tag'])['amount'].sum() * (-1)).rename({'amount': 'transfers_out'}, axis=1).fillna(0)
leverage_acc = leverage_acc.join(latoken_transactions[(latoken_transactions.user_id=='8fdd328f-47cc-4705-bf45-9a4f20314b87')&(latoken_transactions.transaction_type=='DEPOSIT')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'deposits'}, axis=1).fillna(0)
leverage_acc = leverage_acc.join(latoken_transactions[(latoken_transactions.user_id=='8fdd328f-47cc-4705-bf45-9a4f20314b87')&(latoken_transactions.transaction_type=='WITHDRAWAL')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'withdrawals'}, axis=1).fillna(0)
leverage_acc['trade_buy'] = leverage_acc.trade_buy_q + leverage_acc.trade_buy_c
leverage_acc['trade_sell'] = leverage_acc.quantity + leverage_acc.cost
leverage_acc['adj_closing'] = leverage_acc.closing
leverage_acc['adj_opening'] = leverage_acc.opening
leverage_acc['rate_current'] = leverage_acc.index.map(currency_prices_current)
leverage_acc['tier'] = leverage_acc.index.map(token_tier).fillna('tier 3')
leverage_acc = leverage_acc.drop(columns=['quantity', 'cost', 'trade_buy_q', 'trade_buy_c'])
leverage_acc['unexplained_currency'] = leverage_acc.closing - (leverage_acc.opening + leverage_acc.transfers_in +
                                                              leverage_acc.transfers_out + leverage_acc.deposits +
                                                              leverage_acc.withdrawals + leverage_acc.trade_buy +
                                                              leverage_acc.trade_sell)
leverage_acc['unexplained_usd'] = leverage_acc.unexplained_currency * leverage_acc.rate_current
leverage_acc['opening, $'] = leverage_acc.opening * leverage_acc.rate_current
leverage_acc['closing, $'] = leverage_acc.closing * leverage_acc.rate_current
leverage_acc['adj_closing, $'] = leverage_acc.adj_closing * leverage_acc.rate_current
leverage_acc['adj_opening, $'] = leverage_acc.adj_opening * leverage_acc.rate_current
leverage_acc['trade_buy, $'] = leverage_acc.trade_buy * leverage_acc.rate_current
leverage_acc['trade_sell, $'] = leverage_acc.trade_sell * leverage_acc.rate_current
leverage_acc['deposits, $'] = leverage_acc.deposits * leverage_acc.rate_current
leverage_acc['withdrawals, $'] = leverage_acc.withdrawals * leverage_acc.rate_current
leverage_acc['transfers_in, $'] = leverage_acc.transfers_in * leverage_acc.rate_current
leverage_acc['transfers_out, $'] = leverage_acc.transfers_out * leverage_acc.rate_current
leverage_acc['wallet_id'] = '693752'
leverage_acc = leverage_acc[['opening', 'adj_opening', 'opening, $', 'adj_opening, $', 'trade_buy, $', 'trade_sell, $',
                 'deposits, $', 'withdrawals, $','transfers_in, $', 'transfers_out, $', 'closing', 
                 'adj_closing', 'closing, $', 'adj_closing, $', 'unexplained_currency', 'unexplained_usd',
                 'rate_current', 'tier', 'wallet_id']]
eigth = datetime.datetime.now()
print(f'lev done in {eigth-seventh}')

# Internal_mm_LATOKEN
internal_mm_latoken = pd.DataFrame(index = (set(balances_opening[balances_opening.investor_id=='65127'].currency.unique())|set(balances_closing[balances_closing.investor_id=='65127'].currency.unique())))
internal_mm_latoken = internal_mm_latoken.join(balances_closing[balances_closing.investor_id=='65127'].groupby(['currency'])['total'].sum()).rename({'total': 'closing'}, axis=1).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(balances_opening[balances_opening.investor_id=='65127'].groupby(['currency'])['total'].sum()).rename({'total': 'opening'}, axis=1).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(trades[(trades.account=='65127')&(trades.quantity.values>0)].groupby(['currency1'])['quantity'].sum()).rename({'quantity': 'trade_buy_q'}, axis=1).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(trades[(trades.account=='65127')&(trades.cost.values>0)].groupby(['quote1'])['cost'].sum()).rename({'cost': 'trade_buy_c'}, axis=1).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(trades[(trades.account=='65127')&(trades.quantity.values<0)].groupby(['currency1'])['quantity'].sum()).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(trades[(trades.account=='65127')&(trades.cost.values<0)].groupby(['quote1'])['cost'].sum()).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(latoken_transfers[latoken_transfers.to_user_id=='2b58cccc-5fbd-4154-9e73-a009d7c145c9'].groupby(['from_asset_tag'])['amount'].sum()).rename({'amount': 'transfers_in'}, axis=1).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(latoken_transfers[latoken_transfers.from_user_id=='2b58cccc-5fbd-4154-9e73-a009d7c145c9'].groupby(['from_asset_tag'])['amount'].sum() * (-1)).rename({'amount': 'transfers_out'}, axis=1).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(latoken_transactions[(latoken_transactions.user_id=='2b58cccc-5fbd-4154-9e73-a009d7c145c9')&(latoken_transactions.transaction_type=='DEPOSIT')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'deposits'}, axis=1).fillna(0)
internal_mm_latoken = internal_mm_latoken.join(latoken_transactions[(latoken_transactions.user_id=='2b58cccc-5fbd-4154-9e73-a009d7c145c9')&(latoken_transactions.transaction_type=='WITHDRAWAL')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'withdrawals'}, axis=1).fillna(0)
internal_mm_latoken['trade_buy'] = internal_mm_latoken.trade_buy_q + internal_mm_latoken.trade_buy_c
internal_mm_latoken['trade_sell'] = internal_mm_latoken.quantity + internal_mm_latoken.cost
internal_mm_latoken['adj_closing'] = internal_mm_latoken.closing
internal_mm_latoken['adj_opening'] = internal_mm_latoken.opening
internal_mm_latoken['rate_current'] = internal_mm_latoken.index.map(currency_prices_current)
internal_mm_latoken['tier'] = internal_mm_latoken.index.map(token_tier).fillna('tier 3')
internal_mm_latoken = internal_mm_latoken.drop(columns=['quantity', 'cost', 'trade_buy_q', 'trade_buy_c'])
internal_mm_latoken['unexplained_currency'] = internal_mm_latoken.closing - (internal_mm_latoken.opening + internal_mm_latoken.transfers_in +
                                                              internal_mm_latoken.transfers_out + internal_mm_latoken.deposits +
                                                              internal_mm_latoken.withdrawals + internal_mm_latoken.trade_buy +
                                                              internal_mm_latoken.trade_sell)
internal_mm_latoken['unexplained_usd'] = internal_mm_latoken.unexplained_currency * internal_mm_latoken.rate_current
internal_mm_latoken['opening, $'] = internal_mm_latoken.opening * internal_mm_latoken.rate_current
internal_mm_latoken['closing, $'] = internal_mm_latoken.closing * internal_mm_latoken.rate_current
internal_mm_latoken['adj_closing, $'] = internal_mm_latoken.adj_closing * internal_mm_latoken.rate_current
internal_mm_latoken['adj_opening, $'] = internal_mm_latoken.adj_opening * internal_mm_latoken.rate_current
internal_mm_latoken['trade_buy, $'] = internal_mm_latoken.trade_buy * internal_mm_latoken.rate_current
internal_mm_latoken['trade_sell, $'] = internal_mm_latoken.trade_sell * internal_mm_latoken.rate_current
internal_mm_latoken['deposits, $'] = internal_mm_latoken.deposits * internal_mm_latoken.rate_current
internal_mm_latoken['withdrawals, $'] = internal_mm_latoken.withdrawals * internal_mm_latoken.rate_current
internal_mm_latoken['transfers_in, $'] = internal_mm_latoken.transfers_in * internal_mm_latoken.rate_current
internal_mm_latoken['transfers_out, $'] = internal_mm_latoken.transfers_out * internal_mm_latoken.rate_current
internal_mm_latoken['wallet_id'] = '65127'
internal_mm_latoken = internal_mm_latoken[['opening', 'adj_opening', 'opening, $', 'adj_opening, $', 'trade_buy, $', 'trade_sell, $',
                     'deposits, $', 'withdrawals, $','transfers_in, $', 'transfers_out, $', 'closing', 
                     'adj_closing', 'closing, $', 'adj_closing, $', 'unexplained_currency', 'unexplained_usd',
                     'rate_current', 'tier', 'wallet_id']]
ninth = datetime.datetime.now()
print(f'imml done in {ninth-eigth}')
# This section: Pivots all data together
# Creating 'Total' table and adding columns to it
total = pd.concat([internal_mm_other, marginal_acc, leverage_acc, internal_mm_latoken], ignore_index=False, sort=False, axis = 0)
total['adjusted_opening_last_usd'] = total.index.map(currency_prices_last) * total.adj_opening
total['revaluation'] = total['adj_opening, $'] - total.adjusted_opening_last_usd
total['total_change'] = total['adj_closing, $'] - total.adjusted_opening_last_usd
total['internal_trades'] = total['trade_buy, $'] + total['trade_sell, $']
total['net_deposits'] = total['deposits, $'] + total['withdrawals, $']
total['net_transfers'] = total['transfers_in, $'] + total['transfers_out, $']

# Creating 'Exposure' table
exposure = total.groupby([total.index])[['adjusted_opening_last_usd', 'adj_closing, $', 'total_change', 'revaluation', 'internal_trades', 'unexplained_usd', 'net_deposits', 'net_transfers']].sum()
exposure['tier'] = exposure.index.map(token_tier).fillna('tier 3')
exposure = exposure[exposure.tier!='L-tokens']
exposure['exposure, %'] = 0
for i in range(len(exposure)):
    exposure['exposure, %'].iloc[i] = exposure['adj_closing, $'].iloc[i]/exposure['adj_closing, $'].sum()
exposure['exposure, %'] = exposure['exposure, %'] * 100
exposure = exposure.sort_values(by='exposure, %', axis=0, ascending=True)
exposure = exposure.round({'exposure, %': 2, 'adjusted_opening_last_usd': 2, 'adj_closing, $': 2, 
                          'total_change': 2, 'revaluation': 2, 'internal_trades': 0, 'unexplained_usd': 2,
                          'net_deposits': 2, 'net_transfers': 2})

# Adding total heading to the exposure table
exposure_total = pd.DataFrame({'exposure, %': ['Total:', 'Total (excl. LA):', ' '],
                              'adjusted_opening_last_usd': [exposure.adjusted_opening_last_usd.sum(), exposure[exposure.index!='LA'].adjusted_opening_last_usd.sum(), ' '],
                              'adj_closing, $': [exposure['adj_closing, $'].sum(), exposure[exposure.index!='LA']['adj_closing, $'].sum(), ' '],
                              'total_change': [exposure.total_change.sum(), exposure[exposure.index!='LA'].total_change.sum(), ' '],
                              'revaluation': [exposure.revaluation.sum(), exposure[exposure.index!='LA'].revaluation.sum(), ' '],
                              'internal_trades': [exposure.internal_trades.sum(), exposure[exposure.index!='LA'].internal_trades.sum(), ' '],
                              'unexplained_usd': [exposure.unexplained_usd.sum(), exposure[exposure.index!='LA'].unexplained_usd.sum(), ' '],
                              'net_deposits': [exposure.net_deposits.sum(), exposure[exposure.index!='LA'].net_deposits.sum(), ' '],
                              'net_transfers': [exposure.net_transfers.sum(), exposure[exposure.index!='LA'].net_transfers.sum(), ' '],
                              'tier': [' ', ' ', ' ']}).round(2)

# Adding explanation to the exposure table
# Self-charged fees
ld = round((latoken_transactions[latoken_transactions.transaction_type=='DEPOSIT'].currency_tag.map(currency_prices_current) *
    latoken_transactions[latoken_transactions.transaction_type=='DEPOSIT'].fee_value).sum(), 2)
lw = round((latoken_transactions[latoken_transactions.transaction_type=='WITHDRAWAL'].currency_tag.map(currency_prices_current) *
    latoken_transactions[latoken_transactions.transaction_type=='WITHDRAWAL'].fee_value).sum(), 2)
# Binance withdrawal, deposit fees
bw = round(transactions_binance[transactions_binance.type=='Withdraw'].fee.sum(), 2)
bd = round(transactions_binance[transactions_binance.type=='Deposit'].fee.sum(), 2)
feeB = binance_margin_fees(start_date, end_date)
feeB = round(feeB.interest_rfc.sum(), 2)

explanation = pd.DataFrame({
    'revaluation': ['Delta PnL', 'Binance trading fees', 'Binance withdrawal fees', 'Binance deposit fees', 'Binance borrow fees', 'LATOKEN withdrawal fees', 'LATOKEN deposit fees', 'Total', 'Unexplained'],
    'internal_trades': [pnl_analysisGeneral.pnl.sum(), ' ', ' ', ' ', ' ', ' ', ' ',
                        "{:,}".format(pnl_analysisGeneral.pnl.sum().round(2)), 
                        "{:,}".format((exposure_total.internal_trades.iloc[0] - pnl_analysisGeneral.pnl.sum()).round(2))], # Our PnL for all pairs
    'unexplained_usd': [' ', 
                    f'{total_binance_fees}', # Binance trading fees
                    f'{bw}', # Binance withdrawal fees
                    f'{bd}', # Binance deposit fees
                    f'{feeB}', # Binance borrow fees
                    f'{lw}', # LATOKEN withdrawal fees, (*Binance borrow fees - unknown)
                    ' ',
                    f'{"{:,}".format((total_binance_fees+bw+lw+bd+feeB).round(2))}',
                   "{:,}".format((exposure_total.unexplained_usd.iloc[0] - ((total_binance_fees+bw+lw+bd+feeB) * -1)).round(2))],
    'net_deposits': [' ', ' ', ' ', ' ', ' ', ' ', 
                     f'{ld}', # LATOKEN deposit fees (*Deposits manual adjustments, Floating bot tokens)
                     f'{"{:,}".format(round(ld, 2))}',
                    "{:,}".format((exposure_total.net_deposits.iloc[0] - (ld * -1)).round(2))], 
    'net_transfers': [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
})

# Creating a final dataframe
final = pd.DataFrame(columns = ['exposure, %', 'adjusted_opening_last_usd', 'adj_closing, $', 'total_change',
                               'revaluation', 'internal_trades', 'unexplained_usd', 'net_deposits', 
                               'net_transfers', 'tier'])
final = final.append(explanation, ignore_index=True)
final = final.append(exposure_total, ignore_index=True)
final = final.append(exposure, ignore_index=False)
final = final.rename({'adjusted_opening_last_usd': 'adj_opening_prev, $', 'total_change': 'Total_change, $',
                                       'revaluation': 'Revaluation, $', 'internal_trades': 'Trading operating result (excl. fees), $', 
                                       'unexplained_usd': 'Fees paid, $', 'net_deposits': 'Net_deposits, $', 
                                       'net_transfers': 'Net_transfers, $'}, axis = 1)
# final

balances received in 0:00:00.540863
transactions and transfers and adjustments received in 0:01:07.587803
currencies are dealt with in 0:00:00.623844
prices received in 0:09:25.650475
immo done in 0:00:00.076441
mar done in 0:00:00.071392
lev done in 0:00:00.050650
imml done in 0:00:00.078768


In [28]:
# Floating bot aggregation
floating = pd.DataFrame(index = (set(startBalanceFloating[startBalanceFloating.investor_id=='FLOATINGBOT'].currency.unique())
                                  |set(endBalanceFloating[endBalanceFloating.investor_id=='FLOATINGBOT'].currency.unique())))
floating = floating.join(endBalanceFloating[endBalanceFloating.investor_id=='FLOATINGBOT'].groupby(['currency'])['total'].sum()).rename({'total': 'closing'}, axis=1).fillna(0)
floating = floating.join(startBalanceFloating[startBalanceFloating.investor_id=='FLOATINGBOT'].groupby(['currency'])['total'].sum()).rename({'total': 'opening'}, axis=1).fillna(0)
floating = floating.join(otherAccountsAdjustments[(otherAccountsAdjustments.updated>=start_date)&(otherAccountsAdjustments.account=='floating')].groupby(['currency_tag'])['amount'].sum()).rename({'amount': 'adjustments'}, axis=1).fillna(0)
floating = floating.join(trades[(trades.account=='floating')&(trades.quantity.values>0)].groupby(['currency1'])['quantity'].sum()).rename({'quantity': 'trade_buy_q'}, axis=1).fillna(0)
floating = floating.join(trades[(trades.account=='floating')&(trades.cost.values>0)].groupby(['quote1'])['cost'].sum()).rename({'cost': 'trade_buy_c'}, axis=1).fillna(0)
floating = floating.join(trades[(trades.account=='floating')&(trades.quantity.values<0)].groupby(['currency1'])['quantity'].sum()).fillna(0)
floating = floating.join(trades[(trades.account=='floating')&(trades.cost.values<0)].groupby(['quote1'])['cost'].sum()).fillna(0)
floating = floating.join(latoken_transfers[latoken_transfers.to_user_id=='e6fa4889-93ab-4f9b-a773-39f079180706'].groupby(['from_asset_tag'])['amount'].sum()).rename({'amount': 'transfers_in'}, axis=1).fillna(0)
floating = floating.join(latoken_transfers[latoken_transfers.from_user_id=='e6fa4889-93ab-4f9b-a773-39f079180706'].groupby(['from_asset_tag'])['amount'].sum() * (-1)).rename({'amount': 'transfers_out'}, axis=1).fillna(0)
floating = floating.join(latoken_transactions[(latoken_transactions.user_id=='e6fa4889-93ab-4f9b-a773-39f079180706')&(latoken_transactions.transaction_type=='DEPOSIT')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'deposits'}, axis=1).fillna(0)
floating = floating.join(latoken_transactions[(latoken_transactions.user_id=='e6fa4889-93ab-4f9b-a773-39f079180706')&(latoken_transactions.transaction_type=='WITHDRAWAL')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'withdrawals'}, axis=1).fillna(0)
floating['trade_buy'] = floating.trade_buy_q + floating.trade_buy_c
floating['trade_sell'] = floating.quantity + floating.cost
floating = floating.drop(columns=['quantity', 'cost', 'trade_buy_q', 'trade_buy_c'])
floating['unexplained_currency'] = floating.closing - (floating.opening + floating.transfers_in +
                                          floating.transfers_out + floating.deposits +
                                          floating.withdrawals + floating.trade_buy +
                                          floating.trade_sell + floating.adjustments)

floating['rate_current'] = floating.index.map(currency_prices_current)
floating['unexplained_usd'] = floating.unexplained_currency * floating.rate_current
floating['opening, $'] = floating.opening * floating.rate_current
floating['closing, $'] = floating.closing * floating.rate_current
floating['trade_buy, $'] = floating.trade_buy * floating.rate_current
floating['trade_sell, $'] = floating.trade_sell * floating.rate_current
floating['deposits, $'] = floating.deposits * floating.rate_current
floating['withdrawals, $'] = floating.withdrawals * floating.rate_current
floating['transfers_in, $'] = floating.transfers_in * floating.rate_current
floating['transfers_out, $'] = floating.transfers_out * floating.rate_current
floating['adjustments, $'] = floating.adjustments * floating.rate_current
floating['wallet_id'] = 'floating'
floating = floating[['opening', 'opening, $', 'trade_buy, $', 'trade_sell, $',
             'deposits, $', 'withdrawals, $','transfers_in, $', 'transfers_out, $', 'adjustments, $', 'closing', 
             'closing, $', 'unexplained_currency', 'unexplained_usd',
             'rate_current', 'wallet_id']]

floating['opening_last_usd'] = floating.index.map(currency_prices_last) * floating.opening
floating['revaluation'] = floating['opening, $'] - floating.opening_last_usd
floating['total_change'] = floating['closing, $'] - floating.opening_last_usd
floating['internal_trades'] = floating['trade_buy, $'] + floating['trade_sell, $']
floating['net_deposits'] = floating['deposits, $'] + floating['withdrawals, $']
floating['net_transfers'] = floating['transfers_in, $'] + floating['transfers_out, $']
floating['net_adjustments'] = floating['adjustments, $']

# Creating 'Exposure' table
exposureFloating = floating.groupby([floating.index])[['opening_last_usd', 'closing, $', 'total_change',
                                                         'revaluation', 'internal_trades', 'unexplained_usd',
                                                         'net_deposits', 'net_transfers', 'net_adjustments']].sum()
exposureFloating['exposure, %'] = (exposureFloating['closing, $']/exposureFloating['closing, $'].sum()) * 100
exposureFloating = exposureFloating.sort_values(by='exposure, %', axis=0, ascending=True)
exposureFloating = exposureFloating.rename({'opening_last_usd': 'opening, $', 
                                           'internal_trades': 'Trading operating result (excl. fees), $',
                                           'unexplained_usd': 'fees paid, $'}, axis = 1).round(
                         {'exposure, %': 2, 'opening, $': 2, 'closing, $': 2, 
                          'total_change': 2, 'revaluation': 2, 'Trading operating result (excl. fees), $': 0, 
                          'fees paid, $': 2, 'net_deposits': 2, 'net_transfers': 2, 'net_adjustments': 2})
exposureFloating.loc['Total'] = exposureFloating.sum(axis=0)
exposureFloating = pd.concat([exposureFloating[-1:], exposureFloating[:-1]])


# Rotation bot aggregation
rotation = pd.DataFrame(index = (set(balances_opening[balances_opening.investor_id=='ROTATIONBOT'].currency.unique())
                                  |set(balances_closing[balances_closing.investor_id=='ROTATIONBOT'].currency.unique())))
rotation = rotation.join(balances_closing[balances_closing.investor_id=='ROTATIONBOT'].groupby(['currency'])['total'].sum()).rename({'total': 'closing'}, axis=1).fillna(0)
rotation = rotation.join(balances_opening[balances_opening.investor_id=='ROTATIONBOT'].groupby(['currency'])['total'].sum()).rename({'total': 'opening'}, axis=1).fillna(0)
rotation = rotation.join(otherAccountsAdjustments[(otherAccountsAdjustments.updated>=start_date)&(otherAccountsAdjustments.account=='rotation')].groupby(['currency_tag'])['amount'].sum()).rename({'amount': 'adjustments'}, axis=1).fillna(0)
rotation = rotation.join(trades[(trades.account=='rotation')&(trades.quantity.values>0)].groupby(['currency1'])['quantity'].sum()).rename({'quantity': 'trade_buy_q'}, axis=1).fillna(0)
rotation = rotation.join(trades[(trades.account=='rotation')&(trades.cost.values>0)].groupby(['quote1'])['cost'].sum()).rename({'cost': 'trade_buy_c'}, axis=1).fillna(0)
rotation = rotation.join(trades[(trades.account=='rotation')&(trades.quantity.values<0)].groupby(['currency1'])['quantity'].sum()).fillna(0)
rotation = rotation.join(trades[(trades.account=='rotation')&(trades.cost.values<0)].groupby(['quote1'])['cost'].sum()).fillna(0)
rotation = rotation.join(latoken_transfers[latoken_transfers.to_user_id=='fbd3f154-dd62-419d-9d59-bfdc327e231a'].groupby(['from_asset_tag'])['amount'].sum()).rename({'amount': 'transfers_in'}, axis=1).fillna(0)
rotation = rotation.join(latoken_transfers[latoken_transfers.from_user_id=='fbd3f154-dd62-419d-9d59-bfdc327e231a'].groupby(['from_asset_tag'])['amount'].sum() * (-1)).rename({'amount': 'transfers_out'}, axis=1).fillna(0)
rotation = rotation.join(latoken_transactions[(latoken_transactions.user_id=='fbd3f154-dd62-419d-9d59-bfdc327e231a')&(latoken_transactions.transaction_type=='DEPOSIT')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'deposits'}, axis=1).fillna(0)
rotation = rotation.join(latoken_transactions[(latoken_transactions.user_id=='fbd3f154-dd62-419d-9d59-bfdc327e231a')&(latoken_transactions.transaction_type=='WITHDRAWAL')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'withdrawals'}, axis=1).fillna(0)
rotation['trade_buy'] = rotation.trade_buy_q + rotation.trade_buy_c
rotation['trade_sell'] = rotation.quantity + rotation.cost
rotation = rotation.drop(columns=['quantity', 'cost', 'trade_buy_q', 'trade_buy_c'])
rotation['unexplained_currency'] = rotation.closing - (rotation.opening + rotation.transfers_in +
                                          rotation.transfers_out + rotation.deposits +
                                          rotation.withdrawals + rotation.trade_buy +
                                          rotation.trade_sell + rotation.adjustments)

rotation['rate_current'] = rotation.index.map(currency_prices_current)
rotation['unexplained_usd'] = rotation.unexplained_currency * rotation.rate_current
rotation['opening, $'] = rotation.opening * rotation.rate_current
rotation['closing, $'] = rotation.closing * rotation.rate_current
rotation['trade_buy, $'] = rotation.trade_buy * rotation.rate_current
rotation['trade_sell, $'] = rotation.trade_sell * rotation.rate_current
rotation['deposits, $'] = rotation.deposits * rotation.rate_current
rotation['withdrawals, $'] = rotation.withdrawals * rotation.rate_current
rotation['transfers_in, $'] = rotation.transfers_in * rotation.rate_current
rotation['transfers_out, $'] = rotation.transfers_out * rotation.rate_current
rotation['adjustments, $'] = rotation.adjustments * rotation.rate_current
rotation['wallet_id'] = 'rotation'
rotation = rotation[['opening', 'opening, $', 'trade_buy, $', 'trade_sell, $',
             'deposits, $', 'withdrawals, $','transfers_in, $', 'transfers_out, $', 'adjustments, $', 'closing', 
             'closing, $', 'unexplained_currency', 'unexplained_usd',
             'rate_current', 'wallet_id']]

rotation['opening_last_usd'] = rotation.index.map(currency_prices_last) * rotation.opening
rotation['revaluation'] = rotation['opening, $'] - rotation.opening_last_usd
rotation['total_change'] = rotation['closing, $'] - rotation.opening_last_usd
rotation['internal_trades'] = rotation['trade_buy, $'] + rotation['trade_sell, $']
rotation['net_deposits'] = rotation['deposits, $'] + rotation['withdrawals, $']
rotation['net_transfers'] = rotation['transfers_in, $'] + rotation['transfers_out, $']
rotation['net_adjustments'] = rotation['adjustments, $']

# Creating 'Exposure' table
exposureRotation = rotation.groupby([rotation.index])[['opening_last_usd', 'closing, $', 'total_change',
                                                         'revaluation', 'internal_trades', 'unexplained_usd',
                                                         'net_deposits', 'net_transfers', 'net_adjustments']].sum()
exposureRotation['exposure, %'] = (exposureRotation['closing, $']/exposureRotation['closing, $'].sum()) * 100
exposureRotation = exposureRotation.sort_values(by='exposure, %', axis=0, ascending=True)
exposureRotation = exposureRotation.rename({'opening_last_usd': 'opening, $', 
                                           'internal_trades': 'Trading operating result (excl. fees), $',
                                           'unexplained_usd': 'fees paid, $'}, axis = 1).round(
                         {'exposure, %': 2, 'opening, $': 2, 'closing, $': 2, 
                          'total_change': 2, 'revaluation': 2, 'Trading operating result (excl. fees), $': 0, 
                          'fees paid, $': 2, 'net_deposits': 2, 'net_transfers': 2, 'net_adjustments': 2})
exposureRotation.loc['Total'] = exposureRotation.sum(axis=0)
exposureRotation = pd.concat([exposureRotation[-1:], exposureRotation[:-1]])


# LaLiquidity bot aggregation
liquidity = pd.DataFrame(index = (set(balances_opening[balances_opening.investor_id=='LALIQUIDITY'].currency.unique())
                                  |set(balances_closing[balances_closing.investor_id=='LALIQUIDITY'].currency.unique())))
liquidity = liquidity.join(balances_closing[balances_closing.investor_id=='LALIQUIDITY'].groupby(['currency'])['total'].sum()).rename({'total': 'closing'}, axis=1).fillna(0)
liquidity = liquidity.join(balances_opening[balances_opening.investor_id=='LALIQUIDITY'].groupby(['currency'])['total'].sum()).rename({'total': 'opening'}, axis=1).fillna(0)
liquidity = liquidity.join(otherAccountsAdjustments[(otherAccountsAdjustments.updated>=start_date)&(otherAccountsAdjustments.account=='laliquidity')].groupby(['currency_tag'])['amount'].sum()).rename({'amount': 'adjustments'}, axis=1).fillna(0)
liquidity = liquidity.join(trades[(trades.account=='laliquidity')&(trades.quantity.values>0)].groupby(['currency1'])['quantity'].sum()).rename({'quantity': 'trade_buy_q'}, axis=1).fillna(0)
liquidity = liquidity.join(trades[(trades.account=='laliquidity')&(trades.cost.values>0)].groupby(['quote1'])['cost'].sum()).rename({'cost': 'trade_buy_c'}, axis=1).fillna(0)
liquidity = liquidity.join(trades[(trades.account=='laliquidity')&(trades.quantity.values<0)].groupby(['currency1'])['quantity'].sum()).fillna(0)
liquidity = liquidity.join(trades[(trades.account=='laliquidity')&(trades.cost.values<0)].groupby(['quote1'])['cost'].sum()).fillna(0)
liquidity = liquidity.join(latoken_transfers[latoken_transfers.to_user_id=='708b9327-1c3d-4841-8ee4-85acfbbc3159'].groupby(['from_asset_tag'])['amount'].sum()).rename({'amount': 'transfers_in'}, axis=1).fillna(0)
liquidity = liquidity.join(latoken_transfers[latoken_transfers.from_user_id=='708b9327-1c3d-4841-8ee4-85acfbbc3159'].groupby(['from_asset_tag'])['amount'].sum() * (-1)).rename({'amount': 'transfers_out'}, axis=1).fillna(0)
liquidity = liquidity.join(latoken_transactions[(latoken_transactions.user_id=='708b9327-1c3d-4841-8ee4-85acfbbc3159')&(latoken_transactions.transaction_type=='DEPOSIT')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'deposits'}, axis=1).fillna(0)
liquidity = liquidity.join(latoken_transactions[(latoken_transactions.user_id=='708b9327-1c3d-4841-8ee4-85acfbbc3159')&(latoken_transactions.transaction_type=='WITHDRAWAL')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'withdrawals'}, axis=1).fillna(0)
liquidity['trade_buy'] = liquidity.trade_buy_q + liquidity.trade_buy_c
liquidity['trade_sell'] = liquidity.quantity + liquidity.cost
liquidity = liquidity.drop(columns=['quantity', 'cost', 'trade_buy_q', 'trade_buy_c'])
liquidity['unexplained_currency'] = liquidity.closing - (liquidity.opening + liquidity.transfers_in +
                                          liquidity.transfers_out + liquidity.deposits +
                                          liquidity.withdrawals + liquidity.trade_buy +
                                          liquidity.trade_sell + liquidity.adjustments)

liquidity['rate_current'] = liquidity.index.map(currency_prices_current)
liquidity['unexplained_usd'] = liquidity.unexplained_currency * liquidity.rate_current
liquidity['opening, $'] = liquidity.opening * liquidity.rate_current
liquidity['closing, $'] = liquidity.closing * liquidity.rate_current
liquidity['trade_buy, $'] = liquidity.trade_buy * liquidity.rate_current
liquidity['trade_sell, $'] = liquidity.trade_sell * liquidity.rate_current
liquidity['deposits, $'] = liquidity.deposits * liquidity.rate_current
liquidity['withdrawals, $'] = liquidity.withdrawals * liquidity.rate_current
liquidity['transfers_in, $'] = liquidity.transfers_in * liquidity.rate_current
liquidity['transfers_out, $'] = liquidity.transfers_out * liquidity.rate_current
liquidity['adjustments, $'] = liquidity.adjustments * liquidity.rate_current
liquidity['wallet_id'] = 'liquidity'
liquidity = liquidity[['opening', 'opening, $', 'trade_buy, $', 'trade_sell, $',
             'deposits, $', 'withdrawals, $','transfers_in, $', 'transfers_out, $', 'adjustments, $', 'closing', 
             'closing, $', 'unexplained_currency', 'unexplained_usd',
             'rate_current', 'wallet_id']]

liquidity['opening_last_usd'] = liquidity.index.map(currency_prices_last) * liquidity.opening
liquidity['revaluation'] = liquidity['opening, $'] - liquidity.opening_last_usd
liquidity['total_change'] = liquidity['closing, $'] - liquidity.opening_last_usd
liquidity['internal_trades'] = liquidity['trade_buy, $'] + liquidity['trade_sell, $']
liquidity['net_deposits'] = liquidity['deposits, $'] + liquidity['withdrawals, $']
liquidity['net_transfers'] = liquidity['transfers_in, $'] + liquidity['transfers_out, $']
liquidity['net_adjustments'] = liquidity['adjustments, $']

# Creating 'Exposure' table
exposureLiquidity = liquidity.groupby([liquidity.index])[['opening_last_usd', 'closing, $', 'total_change',
                                                         'revaluation', 'internal_trades', 'unexplained_usd',
                                                         'net_deposits', 'net_transfers', 'net_adjustments']].sum()
exposureLiquidity['exposure, %'] = (exposureLiquidity['closing, $']/exposureLiquidity['closing, $'].sum()) * 100
exposureLiquidity = exposureLiquidity.sort_values(by='exposure, %', axis=0, ascending=True)
exposureLiquidity = exposureLiquidity.rename({'opening_last_usd': 'opening, $', 
                                           'internal_trades': 'Trading operating result (excl. fees), $',
                                           'unexplained_usd': 'fees paid, $'}, axis = 1).round(
                         {'exposure, %': 2, 'opening, $': 2, 'closing, $': 2, 
                          'total_change': 2, 'revaluation': 2, 'Trading operating result (excl. fees), $': 0, 
                          'fees paid, $': 2, 'net_deposits': 2, 'net_transfers': 2, 'net_adjustments': 2})

exposureLiquidity.loc['Total'] = exposureLiquidity.sum(axis=0)
exposureLiquidity = pd.concat([exposureLiquidity[-1:], exposureLiquidity[:-1]])

In [29]:
# Creating pivot table by account and tier (with subtotals)
by_account = total[total.tier!='L-tokens'].groupby(['wallet_id', 'tier'])['adj_closing, $'].sum().reset_index()
by_account_lat = by_account[by_account.wallet_id=='65127']
by_account_lat = by_account_lat.append(pd.DataFrame({'wallet_id': [' '], 'tier': ['Subtotal:'], 
                                                     'adj_closing, $': [by_account_lat['adj_closing, $'].sum()]}), 
                                                     ignore_index=True)
by_account_other = by_account[by_account.wallet_id=='binance_trading']
by_account_other = by_account_other.append(pd.DataFrame({'wallet_id': [' '], 'tier': ['Subtotal:'], 
                                                     'adj_closing, $': [by_account_other['adj_closing, $'].sum()]}), 
                                                     ignore_index=True)
by_account_mar = by_account[by_account.wallet_id=='692129']
by_account_mar = by_account_mar.append(pd.DataFrame({'wallet_id': [' '], 'tier': ['Subtotal:'], 
                                                     'adj_closing, $': [by_account_mar['adj_closing, $'].sum()]}), 
                                                     ignore_index=True)
by_account_lev = by_account[by_account.wallet_id=='693752']
by_account_lev = by_account_lev.append(pd.DataFrame({'wallet_id': [' '], 'tier': ['Subtotal:'], 
                                                     'adj_closing, $': [by_account_lev['adj_closing, $'].sum()]}), 
                                                     ignore_index=True)

by_account_total = pd.concat([by_account_lat, by_account_mar, by_account_lev, by_account_other], ignore_index=True, sort=False, axis = 0)
by_account_total = by_account_total.append(pd.DataFrame({'wallet_id': ['Floating'], 'tier': ['Subtotal:'], 
                                                     'adj_closing, $': [exposureFloating['closing, $'].sum()]}), 
                                                     ignore_index=True)
by_account_total = by_account_total.append(pd.DataFrame({'wallet_id': ['Rotation'], 'tier': ['Subtotal:'], 
                                                     'adj_closing, $': [exposureRotation['closing, $'].sum()]}), 
                                                     ignore_index=True)
by_account_total = by_account_total.append(pd.DataFrame({'wallet_id': ['LaLiquidity'], 'tier': ['Subtotal:'], 
                                                     'adj_closing, $': [exposureLiquidity['closing, $'].sum()]}), 
                                                     ignore_index=True)
by_account_total = by_account_total.append(pd.DataFrame({'wallet_id': [' '], 'tier': ['Total'],
                                                         'adj_closing, $': [by_account['adj_closing, $'].sum() +
                                                                           exposureLiquidity['closing, $'].sum() +
                                                                           exposureRotation['closing, $'].sum()]}), # + exposureFloating['closing, $'].sum()
                                                       ignore_index=True)
# Styling 
for i in range(len(by_account_total)):
    by_account_total['adj_closing, $'].iloc[i] = "{:,}".format(round(by_account_total['adj_closing, $'].iloc[i], 2))

# by_account_total

In [30]:
# Creating 'balances' table
balances = total[['tier', 'wallet_id', 'adj_opening', 'adj_opening, $', 'adj_closing', 'adj_closing, $']]
balances['CF_currency'] = None
for i in range(len(balances)):
    if balances.tier.iloc[i] == 'L-tokens':
        balances.CF_currency.iloc[i] = 'L-tokens'
    elif (balances.tier.iloc[i] == 'tier 1') or (balances.index[i] == 'LA') or (balances.index[i] == 'XLM'):
        balances.CF_currency.iloc[i] = balances.index[i]
    else:
        balances.CF_currency.iloc[i] = 'USD'

balances['CF_closing'] = None
for i in range(len(balances)):
    if balances.tier.iloc[i] == 'L-tokens':
        balances.CF_closing.iloc[i] = 0
    elif (balances.tier.iloc[i] == 'tier 1') or (balances.index[i] == 'LA') or (balances.index[i] == 'XLM'):
        balances.CF_closing.iloc[i] = balances.adj_closing.iloc[i]
    else:
        balances.CF_closing.iloc[i] = balances['adj_closing, $'].iloc[i]

balances['mm_perimeter'] = None
balances['legacy_account'] = None
for i in range(len(balances)):
    if balances.wallet_id.iloc[i] == 'binance_trading':
        balances.legacy_account.iloc[i] = '200002'
        balances.mm_perimeter.iloc[i] = 'internal_mm_other'
    elif balances.wallet_id.iloc[i] == '65127':
        balances.legacy_account.iloc[i] = '200001'
        balances.mm_perimeter.iloc[i] = 'internal_mm_latoken'
    elif (balances.wallet_id.iloc[i] == '692129') or (balances.wallet_id.iloc[i] == '693752'):
        balances.legacy_account.iloc[i] = '200003'
        balances.mm_perimeter.iloc[i] = 'marginal_acc'

# Grouping, excluding leverage account and L-tokens
balances_grouped = balances[balances.CF_currency!='L-tokens'].groupby(
    ['legacy_account', 'mm_perimeter', 'CF_currency'])['CF_closing'].sum().reset_index()
balances_grouped['Date'] = end_date.date().isoformat()
balances_grouped['rate_current'] = balances_grouped.CF_currency.map(currency_prices_current).fillna(1)
balances_grouped['Balance, $'] = balances_grouped.rate_current * balances_grouped.CF_closing
balances_grouped = balances_grouped[['Date', 'legacy_account', 'mm_perimeter', 'CF_currency', 'CF_closing', 
                                     'rate_current', 'Balance, $']].round(2)

# Styling 
for i in range(len(balances_grouped)):
    balances_grouped['CF_closing'].iloc[i] = "{:,}".format(balances_grouped['CF_closing'].iloc[i])
    balances_grouped['rate_current'].iloc[i] = "{:,}".format(balances_grouped['rate_current'].iloc[i])
    balances_grouped['Balance, $'].iloc[i] = "{:,}".format(balances_grouped['Balance, $'].iloc[i])

# balances_grouped

In [31]:
# Fees breakdown for CF
# Top deposits
dt = latoken_transactions[(latoken_transactions.transaction_type=='DEPOSIT')&(latoken_transactions.currency_tag.isin(['BTC', 'ETH', 'USDT', 'USDC', 'TRX']))]
dt['fee_usd'] = dt.currency_tag.map(currency_prices_current) * dt.fee_value

# Other deposits
do = latoken_transactions[(latoken_transactions.transaction_type=='DEPOSIT')&(-latoken_transactions.currency_tag.isin(['BTC', 'ETH', 'USDT', 'USDC', 'TRX']))]
do['fee_usd'] = do.currency_tag.map(currency_prices_current) * do.fee_value

# Top withdrawals
wt = latoken_transactions[(latoken_transactions.transaction_type=='WITHDRAWAL')&(latoken_transactions.currency_tag.isin(['BTC', 'ETH', 'USDT', 'USDC', 'TRX']))]
wt['fee_usd'] = wt.currency_tag.map(currency_prices_current) * wt.fee_value

# Top withdrawals
wo = latoken_transactions[(latoken_transactions.transaction_type=='WITHDRAWAL')&(-latoken_transactions.currency_tag.isin(['BTC', 'ETH', 'USDT', 'USDC', 'TRX']))]
wo['fee_usd'] = wo.currency_tag.map(currency_prices_current) * wo.fee_value

fee_breakdown = pd.DataFrame({'Type': ['BTC, ETH, USDC, USDT, TRX deposits', 'Other deposits', 'BTC, ETH, USDC, USDT, TRX withdrawals', 'Other withdrawals'],
                             'Amount': [round(dt.fee_usd.sum(), 2), round(do.fee_usd.sum(), 2), round(wt.fee_usd.sum(), 2), round(wo.fee_usd.sum(), 2)]})
# fee_breakdown

In [32]:
# Saving the report
writer = pd.ExcelWriter(f'{os.getcwd()}/mm_report_{start_date.isoformat()}_to_{end_date.isoformat()}.xlsx')
pnl_analysisGeneral.to_excel(writer, sheet_name='PnLGeneral', index = False)
pnl_outliersGeneral.to_excel(writer, sheet_name='PnL_outliersGeneral', index = False)
pnl_analysisFloating.to_excel(writer, sheet_name='PnLFloating', index = False)
pnl_outliersFloating.to_excel(writer, sheet_name='PnL_outliersFloating', index = False)
pnl_analysisRotation.to_excel(writer, sheet_name='PnLRotation', index = False)
pnl_outliersRotation.to_excel(writer, sheet_name='PnL_outliersRotation', index = False)
pnl_analysisLiquidity.to_excel(writer, sheet_name='PnLLiquidityl', index = False)
pnl_outliersLiquidity.to_excel(writer, sheet_name='PnL_outliersLiquidity', index = False)
fees_paid_binance.to_excel(writer, sheet_name='Binance_fees', index = False)
internal_mm_other.to_excel(writer, sheet_name='internal_mm_other', index = True)
marginal_acc.to_excel(writer, sheet_name='marginal_acc', index = True)
leverage_acc.to_excel(writer, sheet_name='leverage_acc', index = True)
internal_mm_latoken.to_excel(writer, sheet_name='internal_mm_latoken', index = True)
total.to_excel(writer, sheet_name='total', index = True)
final.to_excel(writer, sheet_name='exposure', index = True)
exposureFloating.to_excel(writer, sheet_name='exposureFloating', index = True)
exposureRotation.to_excel(writer, sheet_name='exposureRotation', index = True)
exposureLiquidity.to_excel(writer, sheet_name='exposureLiquidity', index = True)
by_account_total.to_excel(writer, sheet_name='brd_by_account', index = False)
balances_grouped.to_excel(writer, sheet_name='balances', index = False)
latoken_transactions.to_excel(writer, sheet_name='transactionsLat', index = False)
fee_breakdown.to_excel(writer, sheet_name='fees_CF', index = False)
writer.save()

In [33]:
print(f'{datetime.datetime.now() - startingC}')

0:22:03.086980


In [34]:
# # Getting starting balances
# startBalanceFloating = balances_current_floating(start_date)
# endBalanceFloating = balances_current_floating(end_date)

# # Getting transfers and transactions on LATOKEN
# latokenAdjustmentsFloating = transfer_adjustments_floating()
# latokenTransfersFloating = transfers_latoken(start_date, end_date)
# latokenTransactionsFloating = transactions_latoken(start_date, end_date)
# latokenTransactionsFloating.loc[latokenTransactionsFloating[
#     latokenTransactionsFloating.transaction_type=='WITHDRAWAL'].index, 'value'] *= -1

# # Dataframe
# floating = pd.DataFrame(index = (set(startBalanceFloating[startBalanceFloating.investor_id=='FLOATINGBOT'].currency.unique())
#                                  |set(endBalanceFloating[endBalanceFloating.investor_id=='FLOATINGBOT'].currency.unique())))

# # Getting LATOKEN Prices
# start_lat = int(datetime.datetime(start_date.year, start_date.month, 
#          start_date.day, 0, 0).replace(tzinfo=datetime.timezone.utc).timestamp())

# end_lat = int(datetime.datetime(end_date.year, end_date.month, 
#          end_date.day, 23, 59).replace(tzinfo=datetime.timezone.utc).timestamp())

# latPricesClose = dict()
# latPricebsOpen = dict()
# missing = list()

# btcLat = requests.get(f'https://api.latoken.com/v2/tradingview/history?symbol=BTC%2FUSDT&resolution=1h&from={start_lat}&to={end_lat}').json()
# ethLat = requests.get(f'https://api.latoken.com/v2/tradingview/history?symbol=ETH%2FUSDT&resolution=1h&from={start_lat}&to={end_lat}').json()
# trxLat = requests.get(f'https://api.latoken.com/v2/tradingview/history?symbol=TRX%2FUSDT&resolution=1h&from={start_lat}&to={end_lat}').json()

# for i in range(len(floating.index)):
#     if floating[i] == 'USDT':
#         latPricesClose[floating.index[i]] = 1
#         latPricesOpen[floating.index[i]] = 1
#     else:
#         lat = requests.get(f'https://api.latoken.com/v2/tradingview/history?symbol={floating.index[i]}%2FUSDT&resolution=1h&from={start_lat}&to={end_lat}').json()
#         if lat['s'] == 'ok':
#             latPricesClose[floating.index[i]] = float(lat['c'][-1])
#             latPricesOpen[floating.index[i]] = float(lat['c'][0])
#         else:
#             lat = requests.get(f'https://api.latoken.com/v2/tradingview/history?symbol={floating.index[i]}%2FBTC&resolution=1h&from={start_lat}&to={end_lat}').json()
#             if lat['s'] == 'ok':
#                 latPricesClose[floating.index[i]] = float(lat['c'][-1]) * float(btcLat['c'][-1])
#                 latPricesOpen[floating.index[i]] = float(lat['c'][0]) * float(btcLat['c'][0])
#             else:
#                 lat = requests.get(f'https://api.latoken.com/v2/tradingview/history?symbol={floating.index[i]}%2FETH&resolution=1h&from={start_lat}&to={end_lat}').json()
#                 if lat['s'] == 'ok':
#                     latPricesClose[floating.index[i]] = float(lat['c'][-1]) * float(ethLat['c'][-1])
#                     latPricesOpen[floating.index[i]] = float(lat['c'][0]) * float(ethLat['c'][0])
#                 else: 
#                     lat = requests.get(f'https://api.latoken.com/v2/tradingview/history?symbol={floating.index[i]}%2FTRX&resolution=1h&from={start_lat}&to={end_lat}').json()
#                     if lat['s'] == 'ok':
#                         latPricesClose[floating.index[i]] = float(lat['c'][-1]) * float(trxLat['c'][-1])
#                         latPricesOpen[floating.index[i]] = float(lat['c'][0]) * float(trxLat['c'][0])
#                     else:
#                         latPricesClose[floating.index[i]] = 0
#                         latPricesOpen[floating.index[i]] = 0
#                         missing.append(floating.index[i])
                        
# # Floating bot aggregation
# floating = floating.join(endBalanceFloating[endBalanceFloating.investor_id=='FLOATINGBOT'].groupby(['currency'])['total'].sum()).rename({'total': 'closing'}, axis=1).fillna(0)
# floating = floating.join(startBalanceFloating[startBalanceFloating.investor_id=='FLOATINGBOT'].groupby(['currency'])['total'].sum()).rename({'total': 'opening'}, axis=1).fillna(0)
# floating = floating.join(latokenAdjustmentsFloating[latokenAdjustmentsFloating.updated>=start_date].groupby(['currency_tag'])['amount'].sum()).rename({'amount': 'adjustments'}, axis=1).fillna(0)
# floating = floating.join(latoken_trades[(latoken_trades.account=='floating')&(latoken_trades.quantity.values>0)].groupby(['currency1'])['quantity'].sum()).rename({'quantity': 'trade_buy_q'}, axis=1).fillna(0)
# floating = floating.join(latoken_trades[(latoken_trades.account=='floating')&(latoken_trades.cost.values>0)].groupby(['quote1'])['cost'].sum()).rename({'cost': 'trade_buy_c'}, axis=1).fillna(0)
# floating = floating.join(latoken_trades[(latoken_trades.account=='floating')&(latoken_trades.quantity.values<0)].groupby(['currency1'])['quantity'].sum()).fillna(0)
# floating = floating.join(latoken_trades[(latoken_trades.account=='floating')&(latoken_trades.cost.values<0)].groupby(['quote1'])['cost'].sum()).fillna(0)
# floating = floating.join(latokenTransfersFloating[latokenTransfersFloating.to_user_id=='e6fa4889-93ab-4f9b-a773-39f079180706'].groupby(['from_asset_tag'])['amount'].sum()).rename({'amount': 'transfers_in'}, axis=1).fillna(0)
# floating = floating.join(latokenTransfersFloating[latokenTransfersFloating.from_user_id=='e6fa4889-93ab-4f9b-a773-39f079180706'].groupby(['from_asset_tag'])['amount'].sum() * (-1)).rename({'amount': 'transfers_out'}, axis=1).fillna(0)
# floating = floating.join(latokenTransactionsFloating[(latokenTransactionsFloating.user_id=='e6fa4889-93ab-4f9b-a773-39f079180706')&(latoken_transactions.transaction_type=='DEPOSIT')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'deposits'}, axis=1).fillna(0)
# floating = floating.join(latokenTransactionsFloating[(latokenTransactionsFloating.user_id=='e6fa4889-93ab-4f9b-a773-39f079180706')&(latoken_transactions.transaction_type=='WITHDRAWAL')].groupby(['currency_tag'])['value'].sum()).rename({'value': 'withdrawals'}, axis=1).fillna(0)
# floating['trade_buy'] = floating.trade_buy_q + floating.trade_buy_c
# floating['trade_sell'] = floating.quantity + floating.cost
# floating = a.drop(columns=['quantity', 'cost', 'trade_buy_q', 'trade_buy_c'])
# floating['unexplained_currency'] = floating.closing - (floating.opening + floating.transfers_in +
#                                           floating.transfers_out + floating.deposits +
#                                           floating.withdrawals + floating.trade_buy +
#                                           floating.trade_sell + floating.adjustments)

# floating['rate_current'] = floating.index.map(latPricesClose)
# floating['unexplained_usd'] = floating.unexplained_currency * floating.rate_current
# floating['opening, $'] = floating.opening * floating.rate_current
# floating['closing, $'] = floating.closing * floating.rate_current
# floating['trade_buy, $'] = floating.trade_buy * floating.rate_current
# floating['trade_sell, $'] = floating.trade_sell * floating.rate_current
# floating['deposits, $'] = floating.deposits * floating.rate_current
# floating['withdrawals, $'] = floating.withdrawals * floating.rate_current
# floating['transfers_in, $'] = floating.transfers_in * floating.rate_current
# floating['transfers_out, $'] = floating.transfers_out * floating.rate_current
# floating['adjustments, $'] = floating.adjustments * floating.rate_current
# floating['wallet_id'] = 'floating'
# floating = floating[['opening', 'opening, $', 'trade_buy, $', 'trade_sell, $',
#              'deposits, $', 'withdrawals, $','transfers_in, $', 'transfers_out, $', 'adjustments, $', 'closing', 
#              'closing, $', 'unexplained_currency', 'unexplained_usd',
#              'rate_current', 'wallet_id']]

# floating['opening_last_usd'] = floating.index.map(latPricesOpen) * floating.opening
# floating['revaluation'] = floating['opening, $'] - floating.opening_last_usd
# floating['total_change'] = floating['closing, $'] - floating.opening_last_usd
# floating['internal_trades'] = floating['trade_buy, $'] + floating['trade_sell, $']
# floating['net_deposits'] = floating['deposits, $'] + floating['withdrawals, $']
# floating['net_transfers'] = floating['transfers_in, $'] + floating['transfers_out, $']
# floating['net_adjustments'] = floating['adjustments, $']

# # Creating 'Exposure' table
# exposureFloating = floating.groupby([floating.index])[['opening_last_usd', 'closing, $', 'total_change',
#                                                          'revaluation', 'internal_trades', 'unexplained_usd',
#                                                          'net_deposits', 'net_transfers', 'net_adjustments']].sum()
# exposureFloating['exposure, %'] = (exposureFloating['closing, $']/exposureFloating['closing, $'].sum()) * 100
# exposureFloating = exposureFloating.sort_values(by='exposure, %', axis=0, ascending=True)
# exposureFloating = exposureFloating.rename({'opening_last_usd': 'opening, $'}).round({'exposure, %': 2, 'opening, $': 2, 'closing, $': 2, 
#                           'total_change': 2, 'revaluation': 2, 'internal_trades': 0, 'unexplained_usd': 2,
#                           'net_deposits': 2, 'net_transfers': 2, 'net_adjustments': 2})

# exposureFloating.loc['Total'] = exposureFloating.sum(axis=0)
# exposureFloating = pd.concat([exposureFloating[-1:], exposureFloating[:-1]])
# exposureFloating

In [35]:
# # This section: Creating 'trades' table
# # Getting LATOKEN trades data
# latoken_trades = trades_latoken(start_date, end_date)

# # Converting currency ids into tickers
# latoken_currency_list = set(latoken_trades.currency.unique())
# latoken_quote_list = set(latoken_trades.quote.unique())
# latoken_currency_tuple = tuple(set(latoken_currency_list|latoken_quote_list))

# latoken_currencies = currencies(latoken_currency_tuple)
# latoken_currencies.index = latoken_currencies.id
# latoken_currencies = latoken_currencies.drop(columns=['id'])
# latoken_currencies = latoken_currencies.to_dict()['tag']  # {id: tag} excludes Binance currencies not on LATOKEN
# currencies_ids = {y:x for x,y in latoken_currencies.items()}  # {tag: id} excludes Binance currencies not on LATOKEN

# latoken_trades['currency1'] = latoken_trades.currency.apply(lambda x: latoken_currencies[x])
# latoken_trades['quote1'] = latoken_trades.quote.apply(lambda x: latoken_currencies[x])
# latoken_trades['pair'] = latoken_trades.currency1 + latoken_trades.quote1

# # Making all trades directions correct (if we were takers, then the direction is opposite, because direction is set by maker)
# # Adjusting signs (quantity is negative is we buy, cost is negative if we sell)
# latoken_trades['direction'] = latoken_trades[latoken_trades.status=='taker'].direction.replace(['BUY', 'SELL'],['SELL', 'BUY'])
# latoken_trades.loc[latoken_trades[(latoken_trades.direction=='SELL')&(latoken_trades.status=='taker')].index, 'direction'] = 'SELL'

# latoken_trades.loc[latoken_trades[latoken_trades.direction=='BUY'].index, 'cost'] *= -1
# latoken_trades.loc[latoken_trades[latoken_trades.direction=='SELL'].index, 'quantity'] *= -1

# # Adding two required columns to latoken_trades
# latoken_trades['account'] = latoken_trades.trader.map(account_type)

# # This section: Creating a PnL final analysis table
# pnl_analysis = latoken_trades.groupby(['pair', 'currency1', 'quote1'])[['quantity', 'cost']].sum()
# pnl_analysis = pnl_analysis.reset_index()
# pnl_analysis['quantityUSD'] = pnl_analysis.currency1.map(latPricesClose) * pnl_analysis.quantity
# pnl_analysis['costUSD'] = pnl_analysis.quote1.map(latPricesClose) * pnl_analysis.cost
# pnl_analysis['pnl'] = pnl_analysis.quantityUSD + pnl_analysis.costUSD

# # Formatting
# pnl_analysis = pnl_analysis[['pair', 'quantity', 'cost', 'quantityUSD', 'costUSD', 'pnl']]
# pnl_analysis = pnl_analysis.round({'quantity': 2, 'cost': 2, 'quantityUSD': 2, 'costUSD': 2, 'pnl': 2})
# pnl_analysis = pnl_analysis.sort_values(by='pnl', axis=0, ascending=True)

# # Checking for outliers
# pnl_outliers = pnl_analysis[(pnl_analysis.pnl>extreme)|(pnl_analysis.pnl<-extreme)]
# pnl_outliers = pnl_outliers.sort_values(by='pnl', axis=0, ascending=True)

# pnl_analysis

In [36]:
# Total change explained
# explained = (((exposure_total['Total_change, $'].iloc[0] - (exposure_total['Revaluation, $'].iloc[0] + exposure_total['Internal_trades, $'].iloc[0] + 
#  exposure_total['Unexplained, $'].iloc[0] + exposure_total['Net_deposits, $'].iloc[0] + 
#  exposure_total['Net_transfers, $'].iloc[0]))/exposure_total['Total_change, $'].iloc[0]) * 100).round(2)

# print(f"Total change is not explained by {explained}%")

In [37]:
# final[final['Net_transfers, $']!=0]
# final[final['Net_deposits, $']!=0]
# check: transactions_binance, latoken_transactions

In [38]:
# Saving closing balance prices
# if (mode == 'daily') or (mode == 'weekly'):
# pd.DataFrame.from_dict(currency_prices_current, orient='index').to_csv(f'{os.getcwd()}/prices_weekly.csv')
# else:
#     pass